In [ ]:
import matplotlib.tri as tri
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.art3d as art3d
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec

def cbToData(fnames,fname_out, shuffle = 1):
    datasets=[]

    for fname in fnames:
        DATA=np.load(fname)
        DATA64=[DATA[i::64] for i in range(64)]
        x=np.linspace(0,63,64)
        y=np.linspace(0,len(DATA64[0]),len(DATA64[0])+1)

        if shuffle == 0:
            interleave = [i for i in range(0,64)]

        elif shuffle == 1:
            interleave = [8*(i%8)+int(i/8) for i in range(64)] #EVEN_I, EVEN_Q, EVEN_IB, EVEN_QB, ODD_I, ODD_Q, ODD_IB, ODD_QB
            EVEN_I =  interleave[0 : 8]
            EVEN_Q =  interleave[8 :16]
            EVEN_IB = interleave[16:24]
            EVEN_QB = interleave[24:32]
            ODD_I =   interleave[32:40]
            ODD_Q =   interleave[40:48]
            ODD_IB =  interleave[48:56]
            ODD_QB =  interleave[56:64]
            #interleave = EVEN_I + EVEN_Q + EVEN_IB + EVEN_QB + ODD_I + ODD_Q + ODD_IB + ODD_QB
            interleave = EVEN_I + ODD_I + EVEN_Q + ODD_Q + EVEN_IB + ODD_IB + EVEN_QB + ODD_QB
        elif shuffle ==2:
            interleave = [8*(i%8)+int(i/8) for i in range(64)] #EVEN_I, EVEN_Q, EVEN_IB, EVEN_QB, ODD_I, ODD_Q, ODD_IB, ODD_QB
            EVEN_I =  interleave[0 : 8:2] + interleave[0 +1: 8:2]
            EVEN_Q =  interleave[8 :16:2] + interleave[8 +1:16:2]
            EVEN_IB = interleave[16:24:2] + interleave[16+1:24:2]
            EVEN_QB = interleave[24:32:2] + interleave[24+1:32:2]
            ODD_I =   interleave[32:40:2] + interleave[32+1:40:2]
            ODD_Q =   interleave[40:48:2] + interleave[40+1:48:2]
            ODD_IB =  interleave[48:56:2] + interleave[48+1:56:2]
            ODD_QB =  interleave[56:64:2] + interleave[56+1:64:2]
            #interleave = EVEN_I + EVEN_Q + EVEN_IB + EVEN_QB + ODD_I + ODD_Q + ODD_IB + ODD_QB
            interleave = EVEN_I + ODD_I + EVEN_Q + ODD_Q + EVEN_IB + ODD_IB + EVEN_QB + ODD_QB
        elif shuffle ==3:
            interleave = [i for i in range(0,64,2)]+[i for i in range(0+1,64,2)]
        elif shuffle ==4:
            interleave = [i for i in range(0,64,2)]+[i for i in range(0+1,64,2)]
            EVEN_I =  interleave[ 0+0:32:2]
            EVEN_Q =  interleave[ 0+1:32:2]
            EVEN_IB = interleave[ 0+2:32:2]
            EVEN_QB = interleave[ 0+3:32:2]
            ODD_I =   interleave[32+0:64:2]
            ODD_Q =   interleave[32+1:64:2]
            ODD_IB =  interleave[32+2:64:2]
            ODD_QB =  interleave[32+3:64:2]
            interleave = EVEN_I + EVEN_Q + EVEN_IB + EVEN_QB + ODD_I + ODD_Q + ODD_IB + ODD_QB
        DATA1=[]
        for i in range(64):
            xx=[i]*len(DATA64[0])
            DATA1+=([[each[0],each[1],each[2],int(each[0]/8)] for each in zip(xx,y,DATA64[interleave[i]])])

        DATA1=np.array(DATA1)

        dataset = pd.DataFrame({'SubADC':DATA1[:,0],'Time':DATA1[:,1],'Sample':DATA1[:,2],'SubADC_Group':DATA1[:,3]})
        dataset1=dataset.groupby(['Sample','SubADC']).count().reset_index().copy()
        datasets.append(dataset1)
    return datasets




In [ ]:
from matplotlib.colors import LogNorm

def plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=True, zmaxIn=None, zminIn=None, plotBitLevels=False):
    fig = plt.figure(figsize=(32,18))
    plt.tight_layout()
    fname=fnames[0]
    try:
        module = fname.split('module_')[1].split('_')[0]
    except:
        module = 'X'
    try:
        ppm = fname.split('ppm_')[1].split('_')[0]
    except:
        ppm = 'X'
    try:
        description = fname.split('\\')[-1].split('module')[0]
    except:
        description = 'X'
    try:
        skew = fname.split('skewStart_')[1].split('_')[0]
    except:
        skew = 'X'
        
    #max_z=0
    #for i in range(len(datasets)):
    #    max_z = max(max_z,max(datasets[i]['Time'].values))
    
    count=0
    #outer = gridspec.GridSpec(1, len(datasets)+1, wspace=0.2, hspace=0.2, width_ratios=[1]*len(datasets)+[.05])
    outer = gridspec.GridSpec(nrows=2, ncols=len(datasets)+1, wspace=0.2, hspace=0.2, \
                              width_ratios=[1]*len(datasets)+[.05], height_ratios=[.1,9])
    
    ax_out = fig.add_subplot(outer[0,0])
    #ax_out.set_title('aaa',fontsize='x-large')
    ax_out.text(0, 0, 'DESCRIPTION: {} MODULE: {} PPM: {}'.format(description,module,ppm), fontsize=24)

    ax_out.tick_params(labelcolor=(1.,1.,1., 0.0), top='off', bottom='off', left='off', right='off')
    ax_out._frameon = False
    
    for DATASET in datasets:
        z=DATASET['Time']
        zmax=np.amax(z)
    if log == True:
        zmin=1
    else:
        zmin=0
    
    for DATASET in datasets:
        #inner = gridspec.GridSpecFromSubplotSpec(1, 9,
        #                subplot_spec=outer[count], wspace=0.05, hspace=0.1, width_ratios=[0.25]+[1]*8)
        inner = gridspec.GridSpecFromSubplotSpec(1, 9,
                        subplot_spec=outer[1,count], wspace=0.05, hspace=0.1, width_ratios=[0.25]+[1]*8)
        #Add AX For FileName
        ax = fig.add_subplot(inner[0])
        ax.set_ylabel('Sample '+fnames[count])
        ax.tick_params(labelcolor=(1.,1.,1., 0.0), top=False, bottom=False, left=False, right=False)
        ax._frameon = False
        for set in range(8):
            ax = fig.add_subplot(inner[set+1])
            DATASET1 = DATASET.loc[DATASET['SubADC'].isin([i for i in range(set*8,(set+1)*8)])].copy()
            DATASET1.reset_index()
            x=DATASET1['SubADC']
            y=DATASET1['Sample']
            z=DATASET1['Time']

            z2=np.zeros([256,8])
            for index, row in DATASET1.iterrows():
                z2[int(row['Sample'])+128][int(row['SubADC'])-8*set] = int(row['Time'])

            extent=(0, 8, -128, 128)
            cmap = cm.jet

            cmap = cm.jet
            if not plotBitLevels:
                for i in range(-128,128,8):
                    ax.plot([0,0.5],[i,i],'k-',alpha=.3)
                    ax.plot([7.5,8],[i,i],'k-',alpha=.3)
            else:
                for i in range(0,7):
                    ax.plot([0,8],[pow(2,i),pow(2,i)],'k-',alpha=.5)
                    ax.plot([0,8],[-pow(2,i),-pow(2,i)],'k-',alpha=.5)
                
            if log:
                im = plt.imshow(z2, extent=extent, norm=LogNorm(vmin=1e-15, vmax=zmax), cmap=cmap, aspect='auto')
            else:
                for each in range(len(z2)):
                    for each_each in range(len(z2[each])):
                        if int(z2[each][each_each])==0:
                            
                            z2[each][each_each]='NaN' 
                            #print(z2[each][each_each])
                            
                im = plt.imshow(z2, extent=extent, cmap=cmap, aspect='auto')
            if set !=0:
                ax.set_yticks([])
            ax.set_xticks([])

            if zmaxIn==None and zminIn==None:
                plt.clim(zmin, zmax)
            else:
                plt.clim(zminIn, zmaxIn)
            if set in [0,1,4,5,]:
                ax.set_facecolor('lightgray')
            else:
                ax.set_facecolor('whitesmoke')
        count+=1
        
    #inner = gridspec.GridSpecFromSubplotSpec(1, 1,
    #            subplot_spec=outer[len(datasets)])
    inner = gridspec.GridSpecFromSubplotSpec(1, 1,
                subplot_spec=outer[1,len(datasets)])
    ax = fig.add_subplot(inner[0])
    
    plt.colorbar(im,fraction=.25, pad=0.1,ax=None,cax=ax)
    plt.savefig(fname_out.split('.png')[0]+'_shuffle_'+str(shuffle)+'_log_'+str(log)+'.png')
    plt.show()

    


In [ ]:
#AAA=datasets
#plotCaptureBuffer(AAA, fnames,fname_out, shuffle = 1, log=True)


In [ ]:
def fnameToIndex(name):
    ret={}
    ret['fname_prefix']=name.split('_VDD_0V8_')[0].split('/')[-1]
    ret['VDD_0V8']=name.split('_VDD_0V8_')[1].split('_AVDD10_SAR_ADC_')[0]
    ret['AVDD10_SAR_ADC']=name.split('_AVDD10_SAR_ADC_')[1].split('_AVDD10_ADC_')[0]
    ret['AVDD10_ADC']=name.split('_AVDD10_ADC_')[1].split('_AVDD18_ADC_')[0]
    ret['AVDD18_ADC']=name.split('_AVDD18_ADC_')[1].split('_module_')[0]
    ret['module']=name.split('_module_')[1].split('_ppm_')[0]
    ret['ppm']=name.split('_ppm_')[1].split('__skewStart_')[0]
    ret['SkewStart']=name.split('__skewStart_')[1].split('_')[0]
    ret['type']=name.split('__skewStart_'+ret['SkewStart']+'_')[1].split('_')[0]
    ret['TS']=name.split('__skewStart_'+ret['SkewStart']+'_'+ret['type']+'_')[1].split('.npy')[0]
    return ret

def fnameToFFE(name, DATA1):
    ret=fnameToIndex(name)
    DATA = DATA1.copy()
    DATA = DATA.loc[DATA['TS'].isin([ret['TS']])]
    DATA = DATA.loc[DATA['VDD_0V8'].isin([ret['VDD_0V8']])]
    DATA = DATA.loc[DATA['AVDD10_SAR_ADC'].isin([ret['AVDD10_SAR_ADC']])]
    DATA = DATA.loc[DATA['AVDD10_ADC'].isin([ret['AVDD10_ADC']])]
    DATA = DATA.loc[DATA['AVDD18_ADC'].isin([ret['AVDD18_ADC']])]
    DATA = DATA.loc[DATA['module'].isin(['module'+str(ret['module'])])]
    DATA = DATA.loc[DATA['SkewStart'].isin([ret['SkewStart']])]
    FFE=[]
    for i in range(16):
        FFE.append(DATA['FFE_T'+str(i)].values[-1])
    return FFE

import math
def removeOffsetGain(data):
    # Remove Offset and Gain
    dout_corr = np.copy(data)
    N=len(data)
    dc = np.zeros(64, dtype='f')
    gm = np.zeros(64, dtype='f')
    for j in range(64):
        dc[j] = np.mean(data[j::64])
        gm[j] = np.linalg.norm(dout_corr[j::64])/math.sqrt(N/64)
    for j in range(64):
        dout_corr[j::64] = data[j::64]-dc[j]
        dout_corr[j::64] = dout_corr[j::64]*(gm[0]/gm[j])
    return dout_corr


def removeOffsetGainHack(data, par=25, reverse=False, gain=True):
    # Remove Offset and Gain
    dout_corr = np.copy(data)
    N=len(data)
    dc = np.zeros(64, dtype='f')
    gm = np.zeros(64, dtype='f')
    for j in range(64):
        if not reverse:
            dc[j] = np.mean([i if abs(i) >par else 0 for i in data[j::64]])
            if gain:
                gm[j] = np.linalg.norm([i if abs(i) >par else 0 for i in dout_corr[j::64]])/math.sqrt(N/64)
            else:
                gm[j] = np.linalg.norm(dout_corr[j::64])/math.sqrt(N/64)
        if reverse:
            dc[j] = np.mean([i if abs(i) <=par else 0 for i in data[j::64]])
            if gain:
                gm[j] = np.linalg.norm([i if abs(i) <=par else 0 for i in dout_corr[j::64]])/math.sqrt(N/64)
            else:
                gm[j] = np.linalg.norm(dout_corr[j::64])/math.sqrt(N/64)
    for j in range(64):
        dout_corr[j::64] = data[j::64]-dc[j]
        dout_corr[j::64] = dout_corr[j::64]*(gm[0]/gm[j])
    return dout_corr

def ffe(data, coeff):
    d = np.convolve(data, coeff)
    return d



#DATA1=pd.read_csv(r".\CapturesSkew\OVER_VOLTAGE\CRB_Traffic__LOG.csv")
#fnameToFFE(name, DATA1)

In [ ]:
FOLDER = ('.\\Captures_Liang\\')
#FOLDER = ('.\\CapturesSkew\\OVER_VOLTAGE\\')
FOLDER = ('.\\CAPTURES_PVT\\CAPTURES\\')
import glob
#fnames1 = glob.glob(FOLDER+'*module_1*_ADC_2019*.npy')
#fnames2 = glob.glob(FOLDER+'*module_2*_ADC_2019*.npy')
fnames1 = glob.glob(FOLDER+'*.npy')

capPerSample=100
subadcGroups=64
cnt=1
for module in [fnames1]:#,fnames2]:
    minBalAll=0
    maxBalAll=0
    fig = plt.figure(figsize=(20,30))
    plt.tight_layout()
    ax = fig.add_subplot(1,1,1)
    for fname in module:
        if fname.find('_DC_GAIN_CORR')<0:
            capture=np.load(fname)
            for k in range(subadcGroups):            
                balanceList=[]
                minBal=0
                maxBal=0
                for i in range(int(len(capture)/(256*64*capPerSample))):
                    window=capture[(i*256*64*capPerSample)+k:((i+1)*256*64*capPerSample):subadcGroups]
                    balance=np.mean(window)
                    minBal=min(minBal,balance)
                    maxBal=max(maxBal,balance)
                    minBalAll=min(minBalAll,balance)
                    maxBalAll=max(maxBalAll,balance)
                    balanceList.append(balance)
                    #print(i,minBal,maxBal, balance)
                ax.plot(balanceList, label=fname +\
                        'SubADC Group: {} Balance: Avg {} Min {} Max {}'.format(str(k), str(balance),str(minBal),str(maxBal)))
                ax.plot([0,int(len(capture)/(256*64*capPerSample))],[0,0],'k-',alpha=.1)
                ax.set_ylim(-10,10)
                ax.set_title('CB DC Bias over Diffrent Voltage Conditions Module: '+str(cnt) + 'Min {} Max {}'.format(minBalAll,maxBalAll))
                #ax.legend(loc='lower center', bbox_to_anchor=(0.5, -0.05),
                #      fancybox=True, shadow=True, ncol=1)
    plt.savefig(FOLDER+'DC_Offset_Module{}.png'.format(cnt))
    cnt+=1


In [ ]:
FOLDER = ('.\\CAPTURES_PVT\\')
fnames1 = glob.glob(FOLDER+'LOG_PVT_*.csv')
print(fnames1)
fout=open(FOLDER+'LOG_AllParts.csv','w')
for i in range(len(fnames1)):
    fio=open(fnames1[i],'r')
    out=fio.readlines()
    if i==0:
          fout.write(out[0])
    for line in out[1:]:
          fout.write(line)
    fio.close()
fout.close()

In [ ]:
FOLDER = ('.\\CAPTURES_PVT\\CAPTURES\\TT_66_SQUARES\\')
fnames1 = glob.glob(FOLDER+'*.npy')
fnames_out1 = [i.split('.npy')[0]+'_MinMax.png' for i in fnames1]
existing_png= glob.glob(FOLDER+'*.png')

for i in range(len(fnames1)):
    fnames = [fnames1[i]]
    fname_out = fnames_out1[i]
    if not(fname_out.split('.png')[0]+'_shuffle_1_log_False' in existing_png or fname_out.split('.png')[0]+'_shuffle_1_log_True' in existing_png):
        datasets = cbToData(fnames,fname_out, shuffle=1)
        plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=False, zmaxIn=None, zminIn=1,plotBitLevels=True)  
        #plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=False)  
        plt.clf()


In [ ]:
FOLDER = ('.\\CAPTURES_PVT\\CAPTURES\\FFF_53_UNC\\')
fnames1 = glob.glob(FOLDER+'*.npy')

#for fname in fnames1:
#    fig = plt.figure(figsize=(9,5))
#    plt.tight_layout()
#    ax = fig.add_subplot(1,1,1)
#    ax.set_ylim(-128,128)
#    capture = np.load(fname)
#    ax.plot(capture,'.', markersize=1)
#    for i in range(-128,128,8):
#        ax.plot([0,128],[i,i], 'k-', alpha=.1, linewidth=1)
#    plt.savefig(fname.split('.npy')[0]+'.png')

    

capture = np.array([])
for fname in fnames1:
    capture = np.concatenate([capture, np.load(fname)])
print(capture,len(capture),capture.shape)
fig = plt.figure(figsize=(9,5))
plt.tight_layout()
ax = fig.add_subplot(1,1,1)
ax.set_ylim(-128,128)
showLenght=256*64*10#len(capture)#
ax.plot(capture[0:showLenght],'.', markersize=1)
for i in range(0,showLenght,256*64):
    ax.plot([i,i],[-128,128], 'k-', alpha=.1, linewidth=1)
for i in range(-128,128,8):
    ax.plot([0,128],[i,i], 'k-', alpha=.1, linewidth=1)
plt.savefig(FOLDER+'AllUncCaptures.png')

fnames1 = glob.glob(FOLDER+'*.csv')


#for fname in fnames1:
#    DATA=pd.read_csv(fname)
#    ber=DATA['ber'].values
#    fig = plt.figure(figsize=(9,5))
#    plt.tight_layout()
#    ax = fig.add_subplot(1,1,1)
#    ax.set_ylim(1e-9,1e-5)
#    ax.plot(ber,'-')
#    ax.set_yscale('log')
#    plt.savefig(fname.split('.npy')[0]+'.png')
#
#for fname in fnames1:
#    DATA=pd.read_csv(fname)
#    ffe=[]
#    for ffeTap in range(16):
#        ffe.append(DATA['FFE_T'+str(ffeTap)].values)
#    fig = plt.figure(figsize=(9,5))
#    plt.tight_layout()
#    ax = fig.add_subplot(1,1,1)
#    for i,each in zip(range(len(ffe)),ffe):
#        ax.plot(each,'-', label='FFE_'+str(i))
#    ax.legend(loc='best', fancybox=True, shadow=True, ncol=4)                    
#    plt.savefig(fname.split('.npy')[0]+'_FFE.png')

#

In [ ]:
import matplotlib.tri as tri
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.art3d as art3d
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec

import glob
import numpy as np
FOLDER = ('.\\CAPTURES_PVT\\ANDOVER_SQUARES\\')
fname = FOLDER+'SQUARES_ANDOVER_1555425063.78.csv'

fio=open(fname,'r')
CAPTURES=fio.readlines()
fio.close()

header = CAPTURES[0].split(',')
CAPTURES = CAPTURES[1:]
pars=[]
captures=[]
for each in CAPTURES:
    each=each.split(',\n')[0]
    pars.append([each.split(',')[0]] + [float(i) for i in each.split(',')[1:7]])
    captures.append([int(i) for i in each.split(',')[7:]])

DATA_PASS_FAIL_TRAFFIC = pd.read_csv('.\\CAPTURES_PVT\\PassFail.csv')
#int(DATA_PASS_FAIL_TRAFFIC.loc[DATA_PASS_FAIL_TRAFFIC['PART_NO'].isin(['TTT_51'])]['PASS'].values[0])
def partPass(part):
    if int(DATA_PASS_FAIL_TRAFFIC.loc[DATA_PASS_FAIL_TRAFFIC['PART_NO'].isin([part])]['PASS'].values[0])==1:
        return 'PASS'
    else:
        return 'FAIL'

In [ ]:
#PLOT TIME DOMAIN    
conditions = 12
for i in range(142,int(len(captures)/conditions)):
    fig = plt.figure(figsize=(30,30))
    plt.tight_layout()
    ax=[]
    for j in range(12):
        x_range=128
        #plotPos=[0,1,2,3,4,5,6,7,8,9,10,11]
        plotPos=[0,1,2,6,7,8,3,4,5,9,10,11]
        ax.append(fig.add_subplot(int(conditions/3),3,plotPos[j]+1))
        ax[j].set_ylim(-128,128)
        for k in range(-128,128,8):
            ax[j].plot([0,x_range],[k,k], 'k-', alpha=.1, linewidth=1)
        sample = i*conditions + j
        PART_NO = pars[sample][0]
        PASS =  partPass(PART_NO.split('_ITER')[0])
        SUPPLY = pars[sample][1]
        AVDD10_ADC = pars[sample][2]
        TEMP = pars[sample][3]
        avgCap = pars[sample][4]
        minCap = pars[sample][5]
        maxCap = pars[sample][6]
        capture = captures[sample][7:]
        ax[j].set_title('PART_NO {} SUPPLY {} AVDD10_ADC {} TEMP {} avgCap {} minCap {} maxCap {}'.format(PART_NO,SUPPLY,AVDD10_ADC,TEMP,avgCap,minCap,maxCap), fontsize='small')
        ln=[]
        labels=['I_O','Q_O','IB_O','IQ_O','I_E','Q_E','IB_E','IQ_E']
        for subADC in range(8):
            ln.append(ax[j].plot(capture[subADC:x_range*8:8],'.',markersize=5, label=labels[subADC], alpha=.6))
        lnAvg, = ax[j].plot([0,x_range],[avgCap,avgCap], 'r--', alpha=.5, linewidth=1)
        lnMin, = ax[j].plot([0,x_range],[minCap,minCap], 'y--', alpha=.8, linewidth=1)
        lnMax, = ax[j].plot([0,x_range],[maxCap,maxCap], 'y--', alpha=.8, linewidth=1)
        lnZero, = ax[j].plot([0,x_range],[0,0], 'g--', alpha=.8, linewidth=1)
        textstr = 'Traffic {}'.format(PASS)
        props = dict(boxstyle='round', facecolor='white', alpha=0.5)
        ax[j].text(0.05, 0.95, textstr, transform=ax[j].transAxes, fontsize=14, verticalalignment='top', bbox=props)
        ax[j].legend(loc='upper right', ncol=2)
    fig.subplots_adjust(hspace=.2, wspace=0.1)
    fig.savefig(FOLDER+'\\td\\'+PART_NO+'.png', dpi=200)
    plt.show()
    plt.clf()



In [ ]:
#PLOT HISTOGRAMS
conditions = 12
for i in range(0,int(len(captures)/conditions)):
    fig = plt.figure(figsize=(30,30))
    plt.tight_layout()
    ax=[]
    for j in range(12):
        #plotPos=[0,1,2,3,4,5,6,7,8,9,10,11]
        plotPos=[0,1,2,6,7,8,3,4,5,9,10,11]
        ax.append(fig.add_subplot(int(conditions/3),3,plotPos[j]+1))
        ylim=2000
        ax[j].set_ylim(0,ylim)
        #ax[j].set_yscale('symlog')
        sample = i*conditions + j
        PART_NO = pars[sample][0]
        PASS =  partPass(PART_NO.split('_ITER')[0])
        SUPPLY = pars[sample][1]
        AVDD10_ADC = pars[sample][2]
        TEMP = pars[sample][3]
        avgCap = pars[sample][4]
        minCap = pars[sample][5]
        maxCap = pars[sample][6]
        capture = captures[sample][7:]
        ax[j].set_title('PART_NO {} SUPPLY {} AVDD10_ADC {} TEMP {} avgCap {} minCap {} maxCap {}'.format(PART_NO,SUPPLY,AVDD10_ADC,TEMP,avgCap,minCap,maxCap), fontsize='small')
        plt.hist([capture[subADC::8] for subADC in range(8)],bins=256,range=(-128,128), stacked=True, label=['I_O','Q_O','IB_O','IQ_O','I_E','Q_E','IB_E','IQ_E'])
        ax[j].plot([avgCap,avgCap],[0,ylim],'g-', alpha=.8)
        ax[j].plot([minCap,minCap],[0,ylim],'y-', alpha=.8)
        ax[j].plot([maxCap,maxCap],[0,ylim],'y-', alpha=.8)
        for k in range(0,8):
            ax[j].plot([pow(2,k),pow(2,k)],[0,ylim],'k-', alpha=.1, linewidth=1)
            ax[j].plot([-pow(2,k),-pow(2,k)],[0,ylim],'k-', alpha=.1, linewidth=1)
        textstr = 'Traffic {}'.format(PASS)
        props = dict(boxstyle='round', facecolor='white', alpha=0.5)
        ax[j].text(0.05, 0.95, textstr, transform=ax[j].transAxes, fontsize=14, verticalalignment='top', bbox=props)
        ax[j].legend(loc='best')
    fig.subplots_adjust(hspace=.2, wspace=0.1)
    fig.savefig(FOLDER+'\\hist\\'+PART_NO+'_hist.png', dpi=200)
    plt.show()
    plt.clf()


In [ ]:
#PLOT BITS VISUALIZATION TOOL
start=80*12                #12*50
toPrint= len(captures)  #start+12*3#
markers=[':','-','--']
colors =['orange','g','r','b']
for part in range(start,toPrint,12):
    res=[]
    for capture in captures[part:part+12]:
        aaa=[np.binary_repr(i, width=8) for i in capture]
        bbb=[0]*8
        for each in aaa:
            for i in range(8):
                bbb[i]+=int(each[7-i])
        res.append(bbb)
    for log in [True,False]:
        for each,par,i in zip(res,pars[part:part+len(res)], range(len(res))):
            #print(par[0],each)
            PART_NO = par[0]
            PASS =  partPass(PART_NO.split('_ITER')[0])
            if i%12==0:
                fig = plt.figure(figsize=(16,8))
                plt.tight_layout()
                ax = fig.add_subplot(1,1,1)
                ax.set_title(par[0] + ' {Temp Supplies AVDD10_ADC Avg}')
            marker = i%3
            color = int(i/3)
            ax.plot(each, markers[marker], color=colors[color], label=str('{0} {1} {2} {3:6.2f}'.format(par[3],par[1],par[2],par[4])), linewidth=1)

            if i%12==11:
                textstr = 'Traffic {}'.format(PASS)
                props = dict(boxstyle='round', facecolor='white', alpha=0.5)
                ax.text(0.05, 0.25, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
                ax.legend(loc='lower left', ncol=4)
                if log:
                    ax.set_ylim(0,2e4)
                    ax.set_yscale('symlog')
                    plt.savefig(FOLDER+'\\bits\\'+par[0]+'_bits_log.png')
                else:
                    ax.set_ylim(0,2e4)
                    plt.savefig(FOLDER+'\\bits\\'+par[0]+'_bits_lin.png')
                plt.show()
                plt.clf()


In [ ]:
#PLOT BITS VISUALIZATION TOOL
start=0*12                #12*50
toPrint= len(captures)  #start+12*3#
markers=[':','-','--']
colors =['orange','g','r','b']
for part in range(start,toPrint,12):
    res=[]
    for capture in captures[part:part+12]:
        aaa=[np.binary_repr(i, width=8) for i in capture]
        bbb=[0]*8
        for each in aaa:
            for i in range(8):
                bbb[i]+=int(each[7-i])
        ccc=[0]*8
        for i in range(8):
            ccc[i]=sum(bbb[:i+1])
        res.append(ccc)
    for log in [True,False]:
        for each,par,i in zip(res,pars[part:part+len(res)], range(len(res))):
            #print(par[0],each)
            PART_NO = par[0]
            PASS =  partPass(PART_NO.split('_ITER')[0])
            if i%12==0:
                fig = plt.figure(figsize=(16,8))
                plt.tight_layout()
                ax = fig.add_subplot(1,1,1)
                ax.set_title(par[0] + ' {Temp Supplies AVDD10_ADC Avg}')
            marker = i%3
            color = int(i/3)
            ax.plot(each, markers[marker], color=colors[color], label=str('{0} {1} {2} {3:6.2f}'.format(par[3],par[1],par[2],par[4])), linewidth=1)

            if i%12==11:
                textstr = 'Traffic {}'.format(PASS)
                props = dict(boxstyle='round', facecolor='white', alpha=0.5)
                ax.text(0.85, 0.22, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
                ax.legend(loc='lower right', ncol=4)
                for i in range(1,9):
                    ax.plot([i-1,7],[i*256*64/2,i*256*64/2],'k', alpha=1, linewidth=1)
                if log:
                    ax.set_ylim(256*64/2/2,256*64/2*10)
                    ax.set_yscale('symlog')
                    plt.savefig(FOLDER+'\\bitsSum\\'+par[0]+'_bitsSum_log.png')
                else:
                    ax.set_ylim(0,256*64/2*10)
                    plt.savefig(FOLDER+'\\bitsSum\\'+par[0]+'_bitsSum_lin.png')
                plt.show()
                plt.clf()


In [ ]:
[a,b for a,b in zip([1,2],range(2))]

In [ ]:
#PLOT BITS VISUALIZATION FUNCTION

def bitVisData(captures=[], mode=0, NUM_CB=1):
    res=[]
    SAMPLES=NUM_CB*256*64
    for capture in captures:
        aaa=[np.binary_repr(i, width=8) for i in capture]
        bbb=[0]*8
        for each in aaa:
            for i in range(8):
                bbb[i]+=int(each[7-i])
        ccc=[0]*8
        for i in range(8):
            ccc[i]=sum(bbb[:i+1])
        if mode==0:
            res.append(bbb)
        else:
            res.append(ccc)
    ret=[]
    return res

def bitVis(captures=[], labels=[], log=False, mode=0, NUM_CB=1):
    SAMPLES=NUM_CB*256*64

    data=bitVisData(captures=captures, mode=mode, NUM_CB=NUM_CB)
    fig = plt.figure(figsize=(16,8))
    plt.tight_layout(pad=20)
    ax = fig.add_subplot(1,1,1)
    ax.set_title('bitVis')
    for each,label in zip(data,labels):
        ax.plot(each, linewidth=1, label=label)

    for i in range(1,9):
        ax.plot([i-1,7],[i*SAMPLES/2,i*SAMPLES/2],'k', alpha=1, linewidth=1)
        #if mode == 1: 
        #    ax.plot([i-1,7],[i*SAMPLES/2,i*SAMPLES/2],'k', alpha=1, linewidth=1)
        #else:
        #    ax.plot([0,7],[i*SAMPLES/2,i*SAMPLES/2],'k', alpha=1, linewidth=1)
    ax.legend(loc='lower left', bbox_to_anchor= (1,0), ncol=1)
    if log:
        if mode == 1:
            ax.set_ylim(SAMPLES/2/2,SAMPLES/2*10)
        else:
            ax.set_ylim(SAMPLES/2*.9,SAMPLES/2*1.1)
        ax.set_yscale('symlog')
        #plt.savefig(fname.split('.npy')[0]+'_bitsSum_log.png')
    else:
        if mode == 1:
            ax.set_ylim(0,SAMPLES/2*10)
        else:
            ax.set_ylim(SAMPLES/2*.9,SAMPLES/2*1.1)
        #plt.savefig(fname.split('.npy')[0]+'_bitsSum_lin.png')
    fig.subplots_adjust(right=0.5)
    plt.show()
    return fig
    #plt.clf()


In [ ]:
def bitVisDataSubADC(capture, NUM_CB=1, subADC=8, inv = True):
    res=[]
    resDiff=[]
    resBitSum=[]
    resBitSumDiff=[]
    capture=np.array(capture,dtype=int)
    if NUM_CB==None:
        SAMPLES=len(capture)
    else:
        SAMPLES=NUM_CB*256*64
    capture=capture[:SAMPLES]
    for captureSubADC in [capture[i::subADC] for i in range(subADC)]:
        countBits=[0]*8
        for item in [np.binary_repr(i, width=8) for i in captureSubADC]:
            if inv:
                countBits=[countBits[bit] + int(item[bit]) for bit in range(8)]
            else:
                countBits=[countBits[bit] + int(item[7-bit]) for bit in range(8)]
                
        res.append(countBits)
    for each in res:
        resBitSum.append([sum(each[:i+1]) for i in range(8)])
    for each in res:
        resDiff.append([item - int(SAMPLES/2/subADC) for item in each])
    for each in resBitSum:
        resBitSumDiff.append([each[i] - int(SAMPLES/2/subADC)*(i+1) for i in range(8)])
    return [res,resBitSum,resDiff,resBitSumDiff]

def ExtractBitVisData(fnames, NUM_CB=1, prefix='Test', autoLabel=False, subADC=8, splits=2):
    typeRes=['sum','sum_0_n','Delta_sum','Delta_sum_0_n']
    captures=[]
    
    if autoLabel:
        header = 'VDD_0V8,AVDD10_SAR_ADC,AVDD10_ADC,AVDD18_ADC,module,ppm,SkewStart,split,type,subADC,subADC_G,subADC_OE,bit,value,'
    else:
        header = 'fname,split,type,subADC,subADC_G,subADC_OE,bit,value,'

    fname=FOLDER+prefix+'_BitVisData.csv'
    fio=open(fname,'w')
    fio.write(header+'\n')
    fio.close()
    
    if autoLabel:
        labels=[]
        for fname in fnames:
            index=fnameToIndex(fname)
            labels.append([index['VDD_0V8'],index['AVDD10_SAR_ADC'],\
                          index['AVDD10_ADC'],index['AVDD18_ADC'],\
                          index['module'],index['ppm'],index['SkewStart']])
    else:
        labels=[]
        for fname in fnames:
            labels.append([fname.split('.npy')[0]])

    for fname,label in zip(fnames,labels):
        fullCapture = [int(i) for i in np.load(fname)]
        if NUM_CB==None:
            SAMPLES=len(fullCapture)
        else:
            SAMPLES=NUM_CB*256*64
        sampleCapture=fullCapture[:SAMPLES]
        
        for split in range(splits):
            capture = sampleCapture[split*int(SAMPLES/splits):(split+1)*(int(SAMPLES/splits))]
            data = bitVisDataSubADC(capture=capture, NUM_CB=NUM_CB, subADC=subADC)

            fio=open(FOLDER+prefix+'_BitVisData.csv','a+')
            for t in range(len(typeRes)):
                for subAdc in range(subADC):
                    for bit in range(8):
                        items = label+[split,typeRes[t],subAdc,subAdc%4,(subAdc>>2)%2,bit,data[t][subAdc][bit]]
                        for item in items:
                            fio.write(str(item)+',')
                        fio.write('\n')
    fio.close()


In [ ]:
import glob
import numpy as np
FOLDER = ('.\\190416_ADC_SQW\\')
FOLDER = ('.\\CAPTURES_GOOD\\CRB_Liang_114\\')
fnames = glob.glob(FOLDER+'*Type_0*.npy')
#print(fnames)

#res=[]
#for each in np.load(fname):
#    if each !='NaN':
#        res.append(each)
#    np.save(fname,res)

for fname in fnames:
    if fname.find('Multi')<0:
        #[res,resBitSum,resDiff,resBitSumDiff]
        capture=np.load(fname)
        subADC=64
        NUM_CB=None
        result = bitVisDataSubADC(capture, NUM_CB=NUM_CB, subADC=subADC)
        exp_bits = int(len(capture)/2/subADC)
        print(exp_bits,len(capture))
        res=result[0]
        resBitSum=result[1]
        resDiff=result[2]
        resBitSumDiff=result[3]
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)
        plotData=resBitSum

        maxArr=np.amax(plotData)
        ax.plot(plotData,'o-')
        ax.set_title(fname.split('\\')[-1])
        ax.xaxis.set_major_locator(plt.MultipleLocator(4))
        for i in range(64):
            ax.plot([i,i],[0,maxArr],'k-',alpha=.3)
        for i in range(0,64,4):
            ax.plot([i,i],[0,maxArr],'k-',alpha=.3)
        for i in range(0,64,8):
            ax.plot([i,i],[0,maxArr],'k-',alpha=.3)
        for i in range(0,exp_bits*9,int(exp_bits)):
            ax.plot([0,64],[i,i],'k-',alpha=.3)
        ax.set_ylim(0,exp_bits*10)
        fig.savefig(FOLDER+'\\BitVis\\'+fname.split('\\')[-1].split('.npy')[0]+'_BitVisSum.png')
        plt.show()
        plt.clf()

        #ExtractBitVisData(fnames=[fname], NUM_CB=4, prefix='BitVisData_', autoLabel=False, subADC=64)

In [ ]:
import matplotlib.tri as tri
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.art3d as art3d
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.gridspec as gridspec
import glob
import numpy as np

def bitVisDataSubADC(capture, NUM_CB=1, subADC=8, inv = True):
    res=[]
    resDiff=[]
    resBitSum=[]
    resBitSumDiff=[]
    capture=np.array(capture,dtype=int)
    if NUM_CB==None:
        SAMPLES=len(capture)
    else:
        SAMPLES=NUM_CB*256*64
    capture=capture[:SAMPLES]
    for captureSubADC in [capture[i::subADC] for i in range(subADC)]:
        countBits=[0]*8
        for item in [np.binary_repr(i, width=8) for i in captureSubADC]:
            if inv:
                countBits=[countBits[bit] + int(item[bit]) for bit in range(8)]
            else:
                countBits=[countBits[bit] + int(item[7-bit]) for bit in range(8)]
                
        res.append(countBits)
    for each in res:
        resBitSum.append([sum(each[:i+1]) for i in range(8)])
    for each in res:
        resDiff.append([item - int(SAMPLES/2/subADC) for item in each])
    for each in resBitSum:
        resBitSumDiff.append([each[i] - int(SAMPLES/2/subADC)*(i+1) for i in range(8)])
    return [res,resBitSum,resDiff,resBitSumDiff]

def bitVisDataSubAdcPlot(fnames):
    for fname in fnames:
        if 1:#fname.find('Multi')<0:
            #[res,resBitSum,resDiff,resBitSumDiff]
            capture=np.load(fname)
            subADC=64
            NUM_CB=None
            if NUM_CB == None:
                lenCap = len(capture)
            else:
                lenCap = NUM_CB*256*64
            result = bitVisDataSubADC(capture, NUM_CB=NUM_CB, subADC=subADC, inv=False)
            exp_bits = int(len(capture)/2/subADC)
            print(exp_bits,len(capture))
            res=result[0]
            resBitSum=result[1]
            resDiff=result[2]
            resBitSumDiff=result[3]
            cmap = cm.coolwarm#jet
            plotData=resBitSumDiff
            fig = plt.figure(figsize=(16,10))
            plt.tight_layout()

            ax = fig.add_subplot(2,1,1)
            y=[i for i in range(64)]*8
            x=[i for i in range(8)]*64
            #z=np.array(plotData).flatten()
            Z=(np.array(plotData) /[(lenCap/2/64*i+1) for i in range(8)] )*100

            surf = ax.imshow(Z.T, cmap=cmap, vmin=-10, vmax=10, aspect=2,)
            cbar=plt.colorbar(surf,shrink=.4, aspect=5,)
            ax.set_title(fname.split('\\')[-1])
            ax.xaxis.set_major_locator(plt.MultipleLocator(4))
            #ax.set_zlim(-2,2)

            plotDataSubADC = []
            for i in range(8):
                plotDataSubADC = plotDataSubADC + plotData[i::8]
            plotDataSubADC_1 = []
            for i in range(4):
                plotDataSubADC_1 = plotDataSubADC_1 + plotData[i*8:i*8+8] + plotData[i*8+32:i*8+32+8]
                #print(len(plotDataSubADC_1))
            plotDataSubADC=plotDataSubADC_1
            Z=(np.array(plotDataSubADC) /[(lenCap/2/64*i+1) for i in range(8)])*100

            ax = fig.add_subplot(2,1,2)
            surf = ax.imshow(Z.T, cmap=cmap, vmin=-10, vmax=10, aspect=2,)
            cbar=plt.colorbar(surf,shrink=.3, aspect=5,)
            ax.set_title('SubADC ODD/EVEN I_O,I_E,Q_O,Q_E,IB_O,IB_E,QB_O,QB_E')
            ax.xaxis.set_major_locator(plt.MultipleLocator(16))
            #ax.set_zlim(-2,2)

            fig.savefig(FOLDER+'\\BitVis\\'+fname.split('\\')[-1].split('.npy')[0]+'_BitVisDiffIm.png')
            plt.show()
            plt.clf()

            #ExtractBitVisData(fnames=[fname], NUM_CB=4, prefix='BitVisData_', autoLabel=False, subADC=64)
            
FOLDER = ('.\\CapturesSkew\\OVER_VOLTAGE\\')
fnames1 = glob.glob(FOLDER+'*ADC*.npy')
fnames=[]
for fname in fnames1:
    if fname.find('_DC_GAIN_CORR')<0:
        fnames.append(fname)
bitVisDataSubAdcPlot(fnames)


In [ ]:
import glob
import numpy as np
FOLDER = ('.\\190416_ADC_SQW\\')

fnames = glob.glob(FOLDER+'*.npy')

#capture=[]
#for fname in fnames:
#    for each in np.load(fname):
#        #print(1)
#        if each !='NaN':
#            capture.append(each)
#np.save(FOLDER+'AllCapt.npy',capture)

result = bitVisDataSubADC(np.load(FOLDER+'AllCapt.npy'), NUM_CB=None, subADC=64)
res=result[0]
resBitSum=result[1]
resDiff=result[2]
resBitSumDiff=result[3]
fig = plt.figure(figsize=(16,8))
plt.tight_layout(pad=20)
ax = fig.add_subplot(1,1,1)
ax.plot(resBitSum,'o')
maxArr=np.amax(resBitSum)
ax.set_title(fname.split('\\')[-1])
ax.xaxis.set_major_locator(plt.MultipleLocator(4))
for i in range(64):
    ax.plot([i,i],[0,maxArr],'k-',alpha=.3)
#ExtractBitVisData(fnames=[fname], NUM_CB=4, prefix='BitVisData_', autoLabel=False, subADC=64)

In [ ]:
import glob
import numpy as np
FOLDER = ('.\\CapturesSkew\\OVER_VOLTAGE\\')
fnames = glob.glob(FOLDER+'*ADC*.npy' )

ExtractBitVisData(fnames=fnames, NUM_CB=None, prefix='Test', autoLabel=True, subADC=64)

In [ ]:
module=1
import glob
import numpy as np

FOLDER = ('.\\CapturesSkew\\OVER_VOLTAGE\\')
fnames1 = glob.glob(FOLDER+'*module_{}*ADC*.npy'.format(module) )

FOLDER = ('.\\Captures_Liang\\')
fnames1 = glob.glob(FOLDER+'*Type_0*.npy'.format(module) )

fnames=[]
for fname in fnames1:
    if fname.find('_DC_GAIN_CORR')<0:
        fnames.append(fname)
print(len(fnames))
#fnames=fnames[0:2]

def visPlot(module, mode, fnames, NUM_CB=1, prefix='Test'):
    captures=[]
    labels=[]
    for fname in fnames:
        index=fnameToIndex(fname)
        #print(fname,index)
        labels.append('VDD_0V8 {} AVDD10_SAR_ADC {} AVDD10_ADC {} AVDD18_ADC {} module {} ppm {} SkewStart {}'.format(\
               index['VDD_0V8'],index['AVDD10_SAR_ADC'],index['AVDD10_ADC'],index['AVDD18_ADC'],index['module'],index['ppm'],index['SkewStart']))
        captures.append([int(i) for i in np.load(fname)[0:256*64*NUM_CB]])

    fig = bitVis(captures=captures, labels=labels, log=False, mode=mode, NUM_CB=NUM_CB)
    fig.savefig(FOLDER+prefix+'BitVis_Module_{}_Mode_{}.png'.format(module, mode))

def visData(module, mode, fnames, NUM_CB=1, prefix='Test', autoLabel=False):
    captures=[]
    if autoLabel:
        labels=[]
        header = 'VDD_0V8,AVDD10_SAR_ADC,AVDD10_ADC,AVDD18_ADC,module,ppm,SkewStart,b0,b1,b2,b3,b4,b5,b6,b7,'
        for fname in fnames:
            index=fnameToIndex(fname)
            #print(fname,index)
            labels.append([index['VDD_0V8'],index['AVDD10_SAR_ADC'],\
                          index['AVDD10_ADC'],index['AVDD18_ADC'],\
                          index['module'],index['ppm'],index['SkewStart']])
    else:
        labels=[]
        header = 'fname,b0,b1,b2,b3,b4,b5,b6,b7,'
        for fname in fnames:
            labels.append([fname])

    for fname in fnames:
        captures.append([int(i) for i in np.load(fname)[0:256*64*NUM_CB]])

    datas = bitVisData(captures=captures, mode=mode, NUM_CB=NUM_CB)
    fio=open(FOLDER+prefix+'BitVisData_Module_{}_Mode_{}.csv'.format(module, mode),'w')
    fio.write(header+'\n')
    for data,label in zip(datas,labels):
        print(label+data)
        for item in label+data:
            fio.write(str(item)+',')
        fio.write('\n')
    fio.close()
    
data=[]
for module in [1,2]:
        visPlot(module, mode, fnames, NUM_CB=200, prefix='Test')
        visData(module, mode, fnames, NUM_CB=200, prefix='Test')


In [ ]:
'abc'[0]

In [ ]:
import glob
import numpy as np
FOLDER = ('.\\CAPTURES_PVT\\CAPTURES\\FFF_53_UNC\\')
fnames1 = glob.glob(FOLDER+'*.npy')

for fname in fnames1:
    capture = np.load(fname)
    np.savetxt(fname.split('.npy')[0]+'.csv',capture, delimiter=',')


In [ ]:
%matplotlib notebook

FOLDER = ('.\\CAPTURES_PVT\\CAPTURES\\FFF_53_SQUARES\\')
fnames1 = glob.glob(FOLDER+'*Temp_25*.npy')
fnamesOrdered=['']*len(fnames1)

for i in range(len(fnames1)):
    fname=fnames1[i]
    AVDD10_ADC=int(fname.split('AVDD10_ADC_')[1].split('_')[0])
    CNT=int(fname.split('CNT_')[1].split('_')[0])
    fnamesOrdered[(AVDD10_ADC+5)*10+CNT]=fname

fnames1 = glob.glob(FOLDER+'*Temp_85*.npy')
fnamesOrdered85=['']*len(fnames1)

for i in range(len(fnames1)):
    fname=fnames1[i]
    AVDD10_ADC=int(fname.split('AVDD10_ADC_')[1].split('_')[0])
    CNT=int(fname.split('CNT_')[1].split('_')[0])
    fnamesOrdered85[(AVDD10_ADC+5)*10+CNT]=fname

import time
fig = plt.figure(figsize=(9,5))
plt.tight_layout()
ax = fig.add_subplot(1,1,1)
ax.set_ylim(-128,128)
for i in range(-128,128,8):
    ax.plot([0,128],[i,i], 'k-', alpha=.1, linewidth=1)
plt.show
for i in range(len(fnamesOrdered)):
    for j in range(0,1):
        fname=fnamesOrdered[i]
        AVDD10_ADC=int(fname.split('AVDD10_ADC_')[1].split('_')[0])
        CNT=int(fname.split('CNT_')[1].split('_')[0])
        ln, = ax.plot(np.load(fname)[128*j:128*(j+1)],'b.',markersize=3)
        ax.set_ylim(-128,128)
        ax.set_title('AVDD10_ADC MARGIN PCT: {} SAMPLE: {}'.format(AVDD10_ADC,CNT))
        
        fig.canvas.draw()
        time.sleep(.1)
        plt.savefig(FOLDER+'TD\\TT_66_SQUARES_OVER_AVDD10_ADC_PIC_{0:05d}'.format(i))
        ln.remove()
        ax.plot(np.load(fname)[128*j:128*(j+1)],'k.',markersize=1, alpha=.2)
        #plt.cla()


In [ ]:
%matplotlib notebook

FOLDER = ('.\\CAPTURES_PVT\\CAPTURES\\TT_52_SQUARES\\')
fnames1 = glob.glob(FOLDER+'*Temp_25*.npy')
fnamesOrdered=['']*len(fnames1)

for i in range(len(fnames1)):
    fname=fnames1[i]
    AVDD10_ADC=int(fname.split('AVDD10_ADC_')[1].split('_')[0])
    CNT=int(fname.split('CNT_')[1].split('_')[0])
    fnamesOrdered[(AVDD10_ADC+5)*10+CNT]=fname

fnames1 = glob.glob(FOLDER+'*Temp_85*.npy')
fnamesOrdered85=['']*len(fnames1)

for i in range(len(fnames1)):
    fname=fnames1[i]
    AVDD10_ADC=int(fname.split('AVDD10_ADC_')[1].split('_')[0])
    CNT=int(fname.split('CNT_')[1].split('_')[0])
    fnamesOrdered85[(AVDD10_ADC+5)*10+CNT]=fname

import time
fig = plt.figure(figsize=(20,10))
plt.tight_layout()
ax = fig.add_subplot(2,1,1)
ax1 = fig.add_subplot(2,1,2)
fig.subplots_adjust(hspace=.2, wspace=0.0)
ax.set_ylim(-128,128)
ax1.set_ylim(-128,128)
for i in range(-128,128,8):
    ax.plot([0,128],[i,i], 'k-', alpha=.1, linewidth=1)
    ax1.plot([0,128],[i,i], 'k-', alpha=.1, linewidth=1)
ax.plot([0,128],[0,0], 'g-', alpha=.5, linewidth=1)
ax1.plot([0,128],[0,0], 'g-', alpha=.5, linewidth=1)
plt.show
for i in range(len(fnamesOrdered)):
    for j in range(0,1):
        fname=fnamesOrdered[i]
        fname85=fnamesOrdered85[i]
        AVDD10_ADC=int(fname.split('AVDD10_ADC_')[1].split('_')[0])
        CNT=int(fname.split('CNT_')[1].split('_')[0])
        capture=np.load(fname)
        averageCapture=np.mean(capture)
        ln, = ax.plot(capture[128*j:128*(j+1)],'b.',markersize=3)
        lnAvg, = ax.plot([0,128],[averageCapture,averageCapture], 'r--', alpha=.5, linewidth=1)
        capture85=np.load(fname85)
        averageCapture85=np.mean(capture85)
        ln1, = ax1.plot(capture85[128*j:128*(j+1)],'b.',markersize=3)
        lnAvg1, = ax1.plot([0,128],[averageCapture85,averageCapture85], 'r--', alpha=.5, linewidth=1)
        #ax.set_ylim(-128,128)
        ax.set_title('AVDD10_ADC MARGIN PCT: {} SAMPLE: {} TEMP: {}'.format(AVDD10_ADC,CNT, 25))
        ax1.set_title('AVDD10_ADC MARGIN PCT: {} SAMPLE: {} TEMP: {}'.format(AVDD10_ADC,CNT, 85))
        
        fig.canvas.draw()
        time.sleep(.1)
        plt.savefig(FOLDER+'TD\\TT_52_SQUARES_OVER_AVDD10_ADC_PIC_{0:05d}'.format(i))
        ln.remove()
        lnAvg.remove()
        ln1.remove()
        lnAvg1.remove()
        ax.plot(np.load(fname)[128*j:128*(j+1)],'k.',markersize=1, alpha=.2)
        ax1.plot(np.load(fname85)[128*j:128*(j+1)],'k.',markersize=1, alpha=.2)
        #plt.cla()


In [ ]:
for i in range(11):
    print('{0:05d}'.format(i))

In [ ]:
filenames = [f for f in os.listdir(pathIn) if (isfile(join(pathIn, f)))]
filenames[0]

In [ ]:
import imageio
import os
from os.path import isfile, join



def png2gif(pathIn, pathOut, fps, outfname, every=1):
    frame_array = []
    filenames = [f for f in os.listdir(pathIn) if (isfile(join(pathIn, f)))]
    print(len(filenames))
    filenames = [f for f in filenames if f.find('.png')>=0]
    print(len(filenames))
    sortfname = filenames
    # for sorting the file names properly
    #sortfname = filenames.sort(key=lambda x: int(x[-8:]))

    with imageio.get_writer(pathOut+outfname, mode='I', fps=fps) as writer:
        for filename in sortfname[::4]:
            print(pathIn+filename)
            image = imageio.imread(pathIn+filename)
            writer.append_data(image)
    writer.close()


FOLDER = ('.\\CAPTURES_PVT\\CAPTURES\\TT_52_SQUARES\\TD\\')

pathIn=FOLDER
pathOut=FOLDER

outfname='TT_52_SQUARES_every_Other.gif'

fps=3
png2gif(pathIn, pathOut, fps, outfname, every=1)


In [ ]:
import pandas as pd

FOLDER=r'C:\\jupyter\\DATA\ELFR\\ESD\\'
fname='ESD_PARTS.csv'

DATA=pd.read_csv(FOLDER+fname)
fname_out=fname.split('.csv')[0]+'_FORMATTED.csv'
fio=open(FOLDER+fname_out,'w')
fio.write('fname,CAPTURE,\n')
for part in DATA.columns:
    fio.write(part+',')
    for each in [i/2 for i in DATA[part].values]:
        if str(float(each)) != 'nan':
            fio.write(str(int(each))+',')
    fio.write('\n')

fio.close()    


In [ ]:
np.loadtxt('C:\\jupyter\\DATA\\DSP\\Burst_Noise\\A0437_large_Bin6152_burst_77.txt')

In [ ]:
import pandas as pd
import glob

FOLDER=r'C:\\jupyter\\DATA\\DSP\\Screen_debug\\Skew255\\'
fnames = glob.glob(FOLDER+'*.txt')

fname_out='CAPTURES_FORMATTED.csv'
fio=open(FOLDER+fname_out,'w')
fio.write('fname,CAPTURE,\n')
for fname in fnames:
    print(fname)
    fio.write(fname+',')
    DATA = np.loadtxt(fname)
    for each in [i for i in DATA]:
        if str(float(each)) != 'nan':
            fio.write(str(int(each))+',')
    fio.write('\n')

fio.close()    


In [ ]:
import pandas as pd
import seaborn as sns

FOLDER=r'C:\\jupyter\\DATA\ELFR\\ESD\\'
DATA=pd.read_csv(FOLDER+'ESD_PARTS.csv')
#
#FOLDER=r'C:\\jupyter\\DATA\\190416_ADC_SQW\\'
#DATA=pd.read_csv(FOLDER+'190416_ADC_SQW.csv')
#
#
for part in DATA.columns:
    print(part)
    np.save(FOLDER+'ESD_CAP_PART_{}.npy'.format(part),[i/2 for i in DATA[part].values])
#
#FOLDER=r'.\\SC\\with-FW_Fin-MXG_captures\\'
#FOLDER=r'.\\SC\\with-FW_Fin-Prism_captures\\'
#FOLDER=r'.\\SC\\PreFIB\\'
#FOLDER=r'.\\SC\\PostFIB\\'
#fnames = glob.glob(FOLDER+'*.txt')
#for fname in fnames:
#    print(fname)
#    capture=np.loadtxt(fname)
#    print(capture)
#    np.save(fname.split('txt')[0]+'.npy',capture)

if 0:
    fnames1 = glob.glob(FOLDER+'*.npy')
    fnames_out1 = [i.split('.npy')[0]+'_MinMax.png' for i in fnames1]
    existing_png= glob.glob(FOLDER+'*.png')

    for i in range(len(fnames1)):
        fnames = [fnames1[i]]
        fname_out = fnames_out1[i]
        if not(fname_out.split('.png')[0]+'_shuffle_1_log_False' in existing_png or fname_out.split('.png')[0]+'_shuffle_1_log_True' in existing_png):
            datasets = cbToData(fnames,fname_out, shuffle=1)
            plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=False, zmaxIn=None, zminIn=1,plotBitLevels=True)  
            #plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=False)  
            plt.clf()


In [ ]:
DATA.columns

In [ ]:
FOLDERS = '.\\SC\\ADC_Char\\B009_07_2018c\\'
fnames = glob.glob(FOLDER+'*.txt')
for fname in fnames:
    print(fname)
    capture=np.loadtxt(fname)
    print(capture)
    np.save(fname.split('txt')[0]+'.npy',capture)


In [ ]:
import glob
FOLDER=r'.\\SC\\with-FW_Fin-MXG_captures\\'
FOLDER=r'.\\SC\\with-FW_Fin-Prism_captures\\'
#FOLDER=r'.\\SC\\PreFIB\\'
#FOLDER=r'.\\SC\\PostFIB\\'

FOLDER = '.\\SC\\ADC_Char\\\B009_08_2018c\\'
fnames = glob.glob(FOLDER+'*.txt')
for fname in fnames:
    print(fname)
    capture=np.loadtxt(fname)
    print(capture)
    np.save(fname.split('.txt')[0]+'.npy',capture)


#PLOT TIME DOMAIN

fnames = glob.glob(FOLDER+'*.npy')
for fname in fnames:
    captures = [np.load(fname)] 
    avgCap = np.mean(captures[0])
    minCap = np.min(captures[0])
    maxCap = np.max(captures[0])
    conditions = 1
    for i in range(1):
        fig = plt.figure(figsize=(30,30))
        plt.tight_layout()
        ax=[]
        for j in range(1):
            x_range=128
            #plotPos=[0,1,2,3,4,5,6,7,8,9,10,11]
            plotPos=[0,1,2,6,7,8,3,4,5,9,10,11]
            ax=fig.add_subplot(1,1,1)
            ax.set_ylim(-128,128)
            for k in range(-128,128,8):
                ax.plot([0,x_range],[k,k], 'k-', alpha=.1, linewidth=1)
            sample = i*conditions + j
            #PART_NO = pars[sample][0]
            #PASS =  partPass(PART_NO.split('_ITER')[0])
            #SUPPLY = pars[sample][1]
            #AVDD10_ADC = pars[sample][2]
            #TEMP = pars[sample][3]
            capture = captures[sample][7:]
            #ax[j].set_title('PART_NO {} SUPPLY {} AVDD10_ADC {} TEMP {} avgCap {} minCap {} maxCap {}'.format(PART_NO,SUPPLY,AVDD10_ADC,TEMP,avgCap,minCap,maxCap), fontsize='small')
            ln=[]
            labels=['I_O','Q_O','IB_O','IQ_O','I_E','Q_E','IB_E','IQ_E']
            avgSubAdc=[]
            minSubAdc=[]
            maxSubAdc=[]
            for subADC in range(8):
                avgSubAdc.append(np.mean(capture[subADC::8]))
                minSubAdc.append(np.min(capture[subADC::8]))
                maxSubAdc.append(np.max(capture[subADC::8]))
                ln.append(ax.plot(capture[subADC:x_range*8:8],'o',markersize=5, label=labels[subADC], alpha=.8))
            lnAvg, = ax.plot([0,x_range],[avgCap,avgCap], 'r--', alpha=.5, linewidth=1)
            lnMin, = ax.plot([0,x_range],[minCap,minCap], 'y--', alpha=.8, linewidth=1)
            lnMax, = ax.plot([0,x_range],[maxCap,maxCap], 'y--', alpha=.8, linewidth=1)
            lnZero, = ax.plot([0,x_range],[0,0], 'g--', alpha=.8, linewidth=1)
            strAvg=''
            for avg1,min1,max1 in zip(avgSubAdc,minSubAdc,maxSubAdc):
                strAvg+='{0:6.2f} {1:6.2f} {2:6.2f} {3:6.2f}\n'.format(avg1,min1,max1,min1+max1)
            textstr = 'avg:{} min:{} max:{}\n{}'.format(avgCap,minCap,maxCap,strAvg)
            props = dict(boxstyle='round', facecolor='white', alpha=0.5)
            ax.text(0.05, 0.65, textstr, transform=ax.transAxes, fontsize=30, verticalalignment='top', bbox=props)
            ax.set_title(fname, fontsize='large')
            ax.legend(loc='upper right', ncol=2)
        fig.subplots_adjust(hspace=.2, wspace=0.1)
        fig.savefig(fname.split('.npy')[0]+'_td.png', dpi=200)
        plt.show()
        plt.clf()



In [ ]:
FOLDER=r'.\\SC\\with-FW_Fin-Prism_captures\\'
#FOLDER=r'.\\SC\\with-FW_Fin-MXG_captures\\'
#FOLDER=r'.\\SC\\PreFIB\\'
#FOLDER=r'.\\SC\\PostFIB\\'

#PLOT TIME DOMAIN

fnames = glob.glob(FOLDER+'*.npy')
for fname in fnames:
    captures = np.load(fname)
    bits=len(captures)/2/64
    avgCap = np.mean(captures[0])
    minCap = np.min(captures[0])
    maxCap = np.max(captures[0])

    result = bitVisDataSubADC(captures, NUM_CB=None, subADC=64)
    res=result[0]
    resBitSum=result[1]
    resDiff=result[2]
    resBitSumDiff=result[3]
    fig = plt.figure(figsize=(16,8))
    plt.tight_layout(pad=20)
    ax = fig.add_subplot(1,1,1)
    resBitSum1=[]
    for i in range(8):
        resBitSum1=resBitSum1+resBitSum[i::8]
    resBitSum=[]
    for i in range(4):
        resBitSum=resBitSum+resBitSum1[i*8:(i+1)*8]+resBitSum1[32+i*8:32+(i+1)*8]
    ax.plot(resBitSum,'o')
    for i in range(9):
        ax.plot([0,64],[(i+1)*bits,(i+1)*bits])
    maxArr=np.amax(resBitSum)
    textstr = 'avg:{} min:{} max:{}'.format(avgCap,minCap,maxCap)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
    ax.set_title(fname.split('\\')[-1])
    ax.xaxis.set_major_locator(plt.MultipleLocator(4))
    for i in range(64):
        ax.plot([i,i],[0,maxArr],'k-',alpha=.1)
    for i in range(0,64,8):
        ax.plot([i,i],[0,maxArr],'k-',alpha=.3)
    for i in range(0,64,16):
        ax.plot([i,i],[0,maxArr],'k-',alpha=.3)
    fig.savefig(fname.split('.npy')[0]+'_BitVis.png')
    plt.show()
    
        

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import glob
import matplotlib.cm as cm

def cleanCapture(capture):
    captureRaw=capture
    capture=[]
    for each in captureRaw:
        if not np.isnan(each):
            capture.append(each)
    return capture
    
def bitVisDataSubADC(capture, NUM_CB=1, subADC=8, inv = True):
    res=[]
    resDiff=[]
    resBitSum=[]
    resBitSumDiff=[]
    capture=np.array(capture,dtype=int)
    if NUM_CB==None:
        SAMPLES=len(capture)
    else:
        SAMPLES=NUM_CB*256*64
    capture=capture[:SAMPLES]
    for captureSubADC in [capture[i::subADC] for i in range(subADC)]:
        countBits=[0]*8
        for item in [np.binary_repr(i, width=8) for i in captureSubADC]:
            if inv:
                countBits=[countBits[bit] + int(item[bit]) for bit in range(8)]
            else:
                countBits=[countBits[bit] + int(item[7-bit]) for bit in range(8)]
                
        res.append(countBits)
    for each in res:
        resBitSum.append([sum(each[:i+1]) for i in range(8)])
    for each in res:
        resDiff.append([item - int(SAMPLES/2/subADC) for item in each])
    for each in resBitSum:
        resBitSumDiff.append([each[i] - int(SAMPLES/2/subADC)*(i+1) for i in range(8)])
    return [res,resBitSum,resDiff,resBitSumDiff]

def plotPlainTD(ax=None, capture=None, fname=None):
    if ax == None:
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)
    avgCap = np.mean(capture)
    minCap = int(np.min(capture ))
    maxCap = int(np.max(capture ))
    SAMPLES=1024
    ax.set_ylim(-128,128)
    for k in range(-128,128,8):
        ax.plot([0,SAMPLES],[k,k], 'k-', alpha=.1, linewidth=1)
    ln=[]
    ln,    = ax.plot(capture[:SAMPLES],'bo',markersize=5, alpha=.8)
    lnAvg, = ax.plot([0,SAMPLES],[avgCap,avgCap], 'r--', alpha=.5, linewidth=1)
    lnMin, = ax.plot([0,SAMPLES],[minCap,minCap], 'y--', alpha=.8, linewidth=1)
    lnMax, = ax.plot([0,SAMPLES],[maxCap,maxCap], 'y--', alpha=.8, linewidth=1)
    lnZero, = ax.plot([0,SAMPLES],[0,0], 'g--', alpha=.8, linewidth=1)
    textstr = 'avg:{0:6.2f} min:{1:d} max:{2:d}'.format(avgCap,minCap,maxCap)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.65, textstr, transform=ax.transAxes, fontsize='large', verticalalignment='top', bbox=props)
    ax.set_title(fname, fontsize='large')
    ax.yaxis.set_major_locator(plt.MultipleLocator(8))
    ax.xaxis.set_major_locator(plt.MultipleLocator(64))
    if ax == None:
        return fig
    else:
        return ax

def plotTD(ax=None, capture=None, fname=None):
    if ax == None:
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)
    avgCap = np.mean(capture)
    minCap = np.min(capture )
    maxCap = np.max(capture )
        
    x_range=128
    ax.set_ylim(-128,128)
    for k in range(-128,128,8):
        ax.plot([0,x_range],[k,k], 'k-', alpha=.1, linewidth=1)
    ln=[]
    labels=['I_O','Q_O','IB_O','IQ_O','I_E','Q_E','IB_E','IQ_E']
    avgSubAdc=[]
    minSubAdc=[]
    maxSubAdc=[]
    for subADC in range(8):
        avgSubAdc.append(np.mean(capture[subADC::8]))
        minSubAdc.append(np.min(capture[subADC::8]))
        maxSubAdc.append(np.max(capture[subADC::8]))
        ln.append(ax.plot(capture[subADC:x_range*8:8],'o',markersize=5, label=labels[subADC], alpha=.8))
    lnAvg, = ax.plot([0,x_range],[avgCap,avgCap], 'r--', alpha=.5, linewidth=1)
    lnMin, = ax.plot([0,x_range],[minCap,minCap], 'y--', alpha=.8, linewidth=1)
    lnMax, = ax.plot([0,x_range],[maxCap,maxCap], 'y--', alpha=.8, linewidth=1)
    lnZero, = ax.plot([0,x_range],[0,0], 'g--', alpha=.8, linewidth=1)
    strAvg=''
    for avg1,min1,max1 in zip(avgSubAdc,minSubAdc,maxSubAdc):
        strAvg+='{0:6.2f} {1:6.2f} {2:6.2f} {3:6.2f}\n'.format(avg1,min1,max1,min1+max1)
    textstr = 'avg:{0:6.2f} min:{1} max:{2}\n{3}'.format(avgCap,minCap,maxCap,strAvg)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.65, textstr, transform=ax.transAxes, fontsize='large', verticalalignment='top', bbox=props)
    ax.set_title(fname, fontsize='large')
    ax.legend(loc='upper right', ncol=2)
    ax.yaxis.set_major_locator(plt.MultipleLocator(8))
    ax.xaxis.set_major_locator(plt.MultipleLocator(8))
    if ax == None:
        return fig
    else:
        return ax
    
def plotBV(ax=None, capture=None, fname=None):
    if ax == None:
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)

    bits=len(capture)/2/64
    avgCap = np.mean(capture)
    minCap = np.min(capture)
    maxCap = np.max(capture)

    result = bitVisDataSubADC(capture, NUM_CB=None, subADC=64)
    
    res=result[0]
    resBitSum=result[1]
    resDiff=result[2]
    resBitSumDiff=result[3]

    resBitSum1=[]
    for i in range(8):
        resBitSum1=resBitSum1+resBitSum[i::8]
    resBitSum=[]
    for i in range(4):
        resBitSum=resBitSum+resBitSum1[i*8:(i+1)*8]+resBitSum1[32+i*8:32+(i+1)*8]
    plt.gca().set_prop_cycle(None)
    ax.plot(resBitSum,'o')
    plt.gca().set_prop_cycle(None)
    for i in range(1,9):
        ax.plot([0,64],[(i)*bits,(i)*bits])
        
    maxArr=np.amax(resBitSum)
    textstr = 'avg:{0:6.2f} min:{1} max:{2}'.format(avgCap,minCap,maxCap)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
    ax.set_title(fname)
    ax.xaxis.set_major_locator(plt.MultipleLocator(8))
    for i in range(64):
        ax.plot([i,i],[0,maxArr],'k-',alpha=.1)
    for i in range(0,64,8):
        ax.plot([i,i],[0,maxArr],'k--',alpha=.3)
    for i in range(0,64,16):
        ax.plot([i,i],[0,maxArr],'k-',alpha=.6)
    if ax == None:
        return fig
    else:
        return ax

def plotCB(ax=None, capture=None, fname=None, log=False, zmaxIn=None , zminIn=None):
    if ax == None:
        fig = plt.figure(figsize=(16,8))
        plt.tight_layout(pad=20)
        ax = fig.add_subplot(1,1,1)

    avgCap = np.mean(capture)
    minCap = np.min(capture)
    maxCap = np.max(capture)

    z=[]
    for i in range(64):
        z.append(np.histogram(capture[i::64], bins=range(-128,128)))
    z1=[]
    for i in range(8):
        z1=z1+z[i::8]
    z=[]
    for i in range(4):
        z=z+z1[i*8:(i+1)*8]+z1[32+i*8:32+(i+1)*8]

    #y=[i for i in range(64)]*256
    #x=[i for i in range(-128,128)]*64
    #z=np.asarray(z).flatten()
    
    z1=[]
    for i in range(64):
        z1.append(z[i][0])
    z=np.asarray(z1,dtype='float').T
    
    if zmaxIn==None and zminIn==None:
        zmax=np.amax(z)
        zmin=0
        #print(zmin,zmax)
    else:
        zmax=zmaxIn
        zmin=zminIn
        
    if not log:
        for each in range(len(z)):

            for j in range(len(z[each])):
                if int(z[each][j])==0:
                    z[each][j]=np.nan

    extent=(0, 64, -128, 128)
    cmap = cm.jet
    for i in range(-128,128,8):
        ax.plot([0,64],[i,i],'k-',alpha=.3)
    for i in range(0,64,8):
        ax.plot([i,i],[-128,128],'k--',alpha=.3)
    for i in range(0,64,16):
        ax.plot([i,i],[-128,128],'k-',alpha=.6)

    ax.set_facecolor('whitesmoke')
    if log:
        im = plt.imshow(z, extent=extent, norm=LogNorm(vmin=1e-15, vmax=zmax), cmap=cmap, aspect='auto')
    else:
        im = plt.imshow(z, extent=extent, cmap=cmap, aspect='auto', vmin=zmin, vmax=zmax)

    ax.xaxis.set_major_locator(plt.MultipleLocator(8))
    ax.yaxis.set_major_locator(plt.MultipleLocator(8))

    plt.clim(zmin,zmax )
    plt.colorbar(im,ax=ax)
    ax.set_title(fname)
    textstr = 'avg:{0:6.2f} min:{1} max:{2}'.format(avgCap,minCap,maxCap)
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=14, verticalalignment='top', bbox=props)
    if ax == None:
        return fig
    else:
        return ax


def plotAll(fnames):
    for fname in fnames:
        fig = plt.figure(figsize=(32,16))
        plt.tight_layout()
        ax=[]
        for i in range(4):
            ax.append(fig.add_subplot(2,2,i+1))


        capture    =cleanCapture(np.load(fname))
        plotTD(ax=ax[0], capture=capture, fname=fname)
        plotPlainTD(ax=ax[1], capture=capture, fname=fname)
        plotBV(ax=ax[2], capture=capture, fname=fname)
        plotCB(ax=ax[3], capture=capture, fname=fname, log=False)
        fig.savefig(fname.split('.npy')[0]+'.png')
        plt.show()
        plt.clf()
        

FOLDER=r'C:\\jupyter\\DATA\ELFR\\'
fnames=glob.glob(FOLDER+'ADC_SQUARE_CAP*.npy')

plotAll(fnames)

In [ ]:
import glob
FOLDER=r'C:\\jupyter\\DATA\ELFR\\'
fnames=glob.glob(FOLDER+'ADC_SQUARE_CAP*.npy')

fig = plt.figure(figsize=(32,16))
plt.tight_layout(pad=20)
ax=[]
for i in range(4):
    ax.append(fig.add_subplot(2,2,i+1))

fname = fnames[0]
capture    =cleanCapture(np.load(fname))
plotTD(ax=ax[0], capture=capture, fname=fname)
plotPlainTD(ax=ax[1], capture=capture, fname=fname)
plotBV(ax=ax[2], capture=capture, fname=fname)
plotCB(ax=ax[3], capture=capture, fname=fname, log=False)
fig.savefig(fname.split('.npy')[0]+'.png')
plt.show()
plt.clf()


In [ ]:
%matplotlib notebook
import glob
FOLDER=r'C:\\jupyter\\DATA\ELFR\\'
#fnames1 = glob.glob(FOLDER+'ADC_SQUARE_4_15*.npy')
fnames1 = glob.glob(FOLDER+'ADC_SQUARE_CAP*.npy')

import time
ffList=[]
for fname in fnames1:
    ff=fname.split('\\')[-1].split('.npy')[0]
    ffList.append(ff)
#    fig = plt.figure(figsize=(9,5))
#    plt.tight_layout()
#    ax = fig.add_subplot(1,1,1)
#    ax.set_ylim(-128,128)
#    for i in range(-128,128,8):
#        ax.plot([0,128],[i,i], 'k-', alpha=.1, linewidth=1)
#    plt.show
#    capture=np.load(fname)
#    for j in range(int(len(capture)/128)):
#        ln, = ax.plot(capture[128*j:128*(j+1)],'b.',markersize=3)
#        ax.set_ylim(-128,128)
#        ax.set_title('FNAME: {} SAMPLE: {}'.format(fname,j))
#
#        fig.canvas.draw()
#        time.sleep(.1)
#        plt.savefig(FOLDER+'TD\\{0}_PIC_{1:05d}.png'.format(ff,j))
#        ln.remove()
#        ax.plot(np.load(fname)[128*j:128*(j+1)],'k.',markersize=1, alpha=.2)
#                #plt.cla()
#

In [ ]:
ffList

In [ ]:
import imageio
import os
from os.path import isfile, join

FOLDER=r'C:\\jupyter\\DATA\ELFR\\TD\\'
ffList=['ADC_SQUARE_CAP_PART_D2']

def png2gif(pathIn, pathOut, fps, outfname, every=1, prefix='.png'):
    frame_array = []
    filenames = [f for f in os.listdir(pathIn) if (isfile(join(pathIn, f)))]
    print(len(filenames))
    filenames = [f for f in filenames if f.find(prefix+'_')>=0]
    print(len(filenames))
    sortfname = filenames
    # for sorting the file names properly
    #sortfname = filenames.sort(key=lambda x: int(x[-8:]))

    with imageio.get_writer(pathOut+outfname, mode='I', fps=fps) as writer:
        for filename in sortfname[::]:
            print(pathIn+filename)
            image = imageio.imread(pathIn+filename)
            writer.append_data(image)
    writer.close()



pathIn=FOLDER
pathOut=FOLDER

#ff=ffList[0]
for ff in ffList:
    outfname=ff+'_movie.gif'

    fps=10
    png2gif(pathIn, pathOut, fps, outfname, every=1, prefix=ff)


In [ ]:
ffList

In [ ]:
import glob
fnames1 = glob.glob(FOLDER+'*skewStart_0_ADC_2019*.npy')
existing_png= glob.glob(FOLDER+'*.png')
#fnames_out1 = [i.split('.npy')[0]+'.png' for i in fnames1]
fnames_out1 = [i.split('.npy')[0]+'_MinMax.png' for i in fnames1]
for i in range(len(fnames1)):
    fnames = [fnames1[i]]
    fname_out = fnames_out1[i]
    if not(fname_out.split('.png')[0]+'_shuffle_1_log_False' in existing_png or fname_out.split('.png')[0]+'_shuffle_1_log_True' in existing_png):
        datasets = cbToData(fnames,fname_out, shuffle=1)
        plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=False, zmaxIn=1200, zminIn=1)  
        #plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=False)  
        plt.clf()
        #plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=True, zmaxIn=1200, zminIn=1)            
        #plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=True)            
        #plt.clf()

In [ ]:
import glob
fnames1 = glob.glob(FOLDER+'*.npy')
fnames_out1 = [i.split('.npy')[0]+'_DC_GAIN_CORR.npy' for i in fnames1]
for i in range(len(fnames1)):
    if fnames1[i].find('_ADC_2019')>=0  and fnames1[i].find('_DC_GAIN_CORR')<0:
        data = np.load(fnames1[i])
        data_out=removeOffsetGain(data)
        np.save(fnames_out1[i],data_out)
        

In [ ]:
import glob
fnames1 = glob.glob(FOLDER+'*.npy')
fnames_out1 = [i.split('.npy')[0]+'_DC_GAIN_CORR_FFE.npy' for i in fnames1]
for i in range(len(fnames1)):
    if (fnames1[i].find('_ADC_2019')>=0 and fnames1[i].find('_DC_GAIN_CORR')<0 and fnames1[i].find('_DC_GAIN_CORR_FFE')<0):
        DATA1=pd.read_csv(FOLDER+'CRB_Traffic__LOG.csv')
        coeff=fnameToFFE(fnames1[i], DATA1)        
        data = np.load(fnames1[i])
        #data_out=removeOffsetGainHack(data,par=15, reverse=False, gain=False)
        data_out=removeOffsetGainHack(data)
        data_out1=[round(x) for x in ffe(data_out, coeff)]
        np.save(fnames_out1[i],data_out1)
        

In [ ]:
hex(0x80 ^ 0xFF)

In [ ]:
def reverse_bit(num,positions):
    pos = '{:0'+str(positions)+'b}'
    return int(pos.format(num)[::-1], 2)

def u2s_twos_comp(val, bits):
    """computes the 2's complement of int value val"""
    if (val & (1 << (bits - 1))) != 0:  # if sign bit is set e.g., 8bit: 128-255
        val = val - (1 << bits)  # compute negative value
    return val  # return positive value as is


def s2u_twos_comp(val, bits):
    """computes int value of signed int value val"""
    if val < 0: val = (pow(2, bits) - 1) - ~val
    return val


def reverse_nibble(a):

    a=s2u_twos_comp(int(a),8)
    #nibble=int(a)&0x3
    #res=int(a)&0xFC | reverse_bit(nibble,2)
    #return u2s_twos_comp(res, 8)
    #a=s2u_twos_comp(int(a),8)
    #res=int(a)&0xe7 | ((a & 0x10)>>1) | ((a & 0x8)<<1) #| ((a & 0x4)>>2)| ((a & 0x1)<<2)
    res=(a&0xfc | ((a&0x3)^0x3))#int(a)&0x7e | ((a & 0x80)>>7) | ((a & 0x1)<<7) 
    #res=a
    res=int(res)&0xF0 | ((res & 0x8)>>0) | ((res & 0x4)>>0) | ((res & 0x2)>>1) | ((res & 0x1)<<1)
    return u2s_twos_comp(res, 8)


In [ ]:
testDATA=np.load(FOLDER+'CRB_Traffic_Sweep_VDD_0V8_4_AVDD10_SAR_ADC_4_AVDD10_ADC_2_AVDD18_ADC_4_module_1_ppm_0__skewStart_0_ADC_2019.4.8_15.16.16.npy')
#testDATA_reversedNibble=[reverse_nibble(i) if j%16>=8 else i for i,j in zip(testDATA,range(len(testDATA)))]
testDATA_reversedNibble=[testDATA[j] if j%8>=4 else testDATA[j+4] for j in range(len(testDATA-16))]

In [ ]:
np.save(FOLDER+'CRB_Traffic_Sweep_VDD_0V8_4_AVDD10_SAR_ADC_4_AVDD10_ADC_2_AVDD18_ADC_4_module_1_ppm_0__skewStart_0_ADC_2019.4.8_15.16.16_REVERSED_NIBBLE.npy',testDATA_reversedNibble)

In [ ]:
testDATA=np.load(FOLDER+'CRB_Traffic_Sweep_VDD_0V8_4_AVDD10_SAR_ADC_4_AVDD10_ADC_2_AVDD18_ADC_4_module_1_ppm_0__skewStart_0_ADC_2019.4.8_15.16.16.npy')
np.save(FOLDER+'CRB_Traffic_Sweep_VDD_0V8_4_AVDD10_SAR_ADC_4_AVDD10_ADC_2_AVDD18_ADC_4_module_1_ppm_0__skewStart_0_ADC_2019.4.8_15.16.16_UNREVERSED_NIBBLE.npy',testDATA)

In [ ]:
import glob
FOLDER=r'.\\TP100C_RevA_SIN\\Chip1_SigGen5\\'

fnames = glob.glob(FOLDER+'*Power*.csv')
DATA=[]
for each in fnames:
    DATA+=[np.loadtxt(each)]
np.save(FOLDER+'PowerAllCaptures.npy',DATA)


In [ ]:
fname=(r".\TP100C_RevA_SIN\Chip1_SigGen5\Chip1_test_SG5_Lane1_Power2_Frequency343.037_lane1_20171213_111727.csv")
np.save(fname.split('.csv')[0]+'.npy',np.loadtxt(fname))
fname=fname.split('.csv')[0]+'.npy'
fname_out = fname.split('.csv')[0]+'.png' 

data = np.load(fname)
data_out=removeOffsetGain(data)
np.save(fname.split('.csv')[0]+'_DC_GAIN_CORR.npy',data_out)
fnames=[fname,fname.split('.csv')[0]+'_DC_GAIN_CORR.npy']

In [ ]:
#fnames=[r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCaptures.npy"]
#fnames=[r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCapturesRevBits012.npy"]
fnames=[r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCapturesRandomFlipBits01234_25pct___.npy"]


fname_out = fnames[0].split('.npy')[0]+'.png' 
for fname in fnames:
    dataset = cbToData([fname],fname_out, shuffle = 1)
    plotCaptureBuffer(dataset, [fname],fname_out, shuffle = 1, log=False)  
    plt.clf()
    plotCaptureBuffer(dataset, [fname],fname_out, shuffle = 1, log=True)            
    plt.clf()

In [ ]:
fname=(r".\TP100C_RevA_SIN\Chip1_SigGen5\Power2Captures.npy")
fname=(r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCaptures.npy")
data1=[]
data= np.load(fname)
for each in data:
    data1= np.concatenate([data1,each])
data=data1


In [ ]:
import random
int(round(random.random()))

In [ ]:
rr=rrBit()
rr_=(rr+1)%2
print(rr,rr_)

In [ ]:
def rrBit():
    return int(round(random.random()))


In [ ]:
np.save(r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCaptures.npy",data1)

In [ ]:
import random
def rrBit():
    return int(round(random.random()))

def reverse_nibble(a):

    a=s2u_twos_comp(int(a),8)
    #nibble=int(a)&0x3
    #res=int(a)&0xFC | reverse_bit(nibble,2)
    #return u2s_twos_comp(res, 8)
    #a=s2u_twos_comp(int(a),8)
    #res=int(a)&0xe7 | ((a & 0x10)>>1) | ((a & 0x8)<<1) #| ((a & 0x4)>>2)| ((a & 0x1)<<2)
    res=(a&0xfc | ((a&0x3)^0x3))#int(a)&0x7e | ((a & 0x80)>>7) | ((a & 0x1)<<7) 
    #res=a
    rr=rrBit()
    if random.random()<0.25 and (res & 0x1F ) == 0xF:
        res=((res & 0x80)>>0) | ((res & 0x40)>>0) | ((res & 0x20)<<0) | ((res ^ 0x10)>>0) | \
            ((res ^ 0x8 )<<0) | ((res ^ 0x4 )>>0) | ((res ^ 0x2 )<<0) | ((res ^ 0x1 )<<0)
    else:
        res=((res & 0x80)>>0) | ((res & 0x40)>>0) | ((res & 0x20)<<0) | ((res & 0x10)>>0) | \
            ((res & 0x8)<<0) | ((res & 0x4)>>0) | ((res & 0x2)<<0) | ((res & 0x1)<<0)
    if random.random()<0.50 and (res & 0xF ) == 0x7:
        res=((res & 0x80)>>0) | ((res & 0x40)>>0) | ((res & 0x20)<<0) | ((res & 0x10)>>0) | \
            ((res ^ 0x8 )<<0) | ((res ^ 0x4 )>>0) | ((res ^ 0x2 )<<0) | ((res ^ 0x1 )<<0)
    else:
        res=((res & 0x80)>>0) | ((res & 0x40)>>0) | ((res & 0x20)<<0) | ((res & 0x10)>>0) | \
            ((res & 0x8)<<0) | ((res & 0x4)>>0) | ((res & 0x2)<<0) | ((res & 0x1)<<0)
    if random.random()<0.25 and (res & 0x7 ) == 0x3:
        res=((res & 0x80)>>0) | ((res & 0x40)>>0) | ((res & 0x20)<<0) | ((res & 0x10)>>0) | \
            ((res & 0x8 )<<0) | ((res ^ 0x4 )>>0) | ((res ^ 0x2 )<<0) | ((res ^ 0x1 )<<0)
    else:
        res=((res & 0x80)>>0) | ((res & 0x40)>>0) | ((res & 0x20)<<0) | ((res & 0x10)>>0) | \
            ((res & 0x8)<<0) | ((res & 0x4)>>0) | ((res & 0x2)<<0) | ((res & 0x1)<<0)
    if random.random()<0.25 and (res & 0x3 ) == 0x1:
        res=((res & 0x80)>>0) | ((res & 0x40)>>0) | ((res & 0x20)<<0) | ((res & 0x10)>>0) | \
            ((res & 0x8 )<<0) | ((res & 0x4 )>>0) | ((res ^ 0x2 )<<0) | ((res ^ 0x1 )<<0)
    else:
        res=((res & 0x80)>>0) | ((res & 0x40)>>0) | ((res & 0x20)<<0) | ((res & 0x10)>>0) | \
            ((res & 0x8)<<0) | ((res & 0x4)>>0) | ((res & 0x2)<<0) | ((res & 0x1)<<0)
    return u2s_twos_comp(res, 8)



In [ ]:
dataR = [reverse_nibble(i) for i in data]

In [ ]:
np.save(r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCapturesRandomFlipBits01234_25pct___.npy",dataR)

In [ ]:
np.save(r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCaptures.npy",data)

In [ ]:
data = data1
#data = [reverse_nibble(i) for i in data]
fig = plt.figure(figsize=(16,8))
plt.tight_layout()
for i in range(64):
    fig = plt.figure(figsize=(16,8))
    plt.tight_layout()
    ax = fig.add_subplot(1,1,1)
    #ax.plot(data[i::8],'-')
    plt.hist(data[i::64],bins=256,range=(-128,128))
    for j in range(-128,128+1,8):
        ax.plot([j,j],[0,5],'k-', alpha=.1)
        plt.show()
        plt.clf()


In [ ]:
data = np.load(r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCaptures.npy")
#data = [reverse_nibble(i) for i in data]
fig = plt.figure(figsize=(16,8))
plt.tight_layout()
for i in range(64):
    fig = plt.figure(figsize=(16,8))
    plt.tight_layout()
    ax = fig.add_subplot(1,1,1)
    #ax.plot(data[i::8],'-')
    plt.hist(data[i::64],bins=256,range=(-128,128))
    for j in range(-128,128+1,8):
        ax.plot([j,j],[0,5],'k-', alpha=.1)
        plt.show()
        plt.clf()


In [ ]:
data = np.load(r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCapturesRandomFlipBits01234_25pct___.npy")
#data = [reverse_nibble(i) for i in data]
fig = plt.figure(figsize=(16,8))
plt.tight_layout()
for i in range(64):
    fig = plt.figure(figsize=(16,8))
    plt.tight_layout()
    ax = fig.add_subplot(1,1,1)
    #ax.plot(data[i::8],'-')
    plt.hist(data[i::64],bins=256,range=(-128,128))
    for j in range(-128,128+1,8):
        ax.plot([j,j],[0,5],'k-', alpha=.1)
        plt.show()
        plt.clf()


In [ ]:
data = np.load(r".\TP100C_RevA_SIN\Chip1_SigGen5\PowerAllCapturesRandomFlipBits01234_25pct_.npy")
#data = [reverse_nibble(i) for i in data]
fig = plt.figure(figsize=(16,8))
plt.tight_layout()
for i in range(64):
    fig = plt.figure(figsize=(16,8))
    plt.tight_layout()
    ax = fig.add_subplot(1,1,1)
    #ax.plot(data[i::8],'-')
    plt.hist(data[i::64],bins=256,range=(-128,128))
    for j in range(-128,128+1,8):
        ax.plot([j,j],[0,5],'k-', alpha=.1)
        plt.show()
        plt.clf()


In [ ]:
TS='_2019.3.29_22.5.24'
prefix_fname = r".\CapturesSkew\2019.3.29_22.5.24\CRB_Traffic_AVDD10_SAR_ADC_AVDD10_ADC_5pct__module_"
               
for ppm in [0,50,100]:
    for module in [1,2]:
        for type in ['DSP','ADC','AFE']:
            fnames=[]
            for skew in [0,127,248]:
                fnames.append(prefix_fname+str(module)+"_ppm_"+str(ppm)+\
                              "__skewStart_"+str(skew)+"_"+str(type)+str(TS)+".npy")

            fname_out=prefix_fname+str(module)+"_ppm_"+str(ppm)+\
                              "_"+str(type)+str(TS)+".png"
            
            datasets = cbToData(fnames,fname_out, shuffle=1)
            plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=False)  
            plt.clf()
            plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=True)            
            plt.clf()

In [ ]:
TS='_2019.3.28_13.37.2'
prefix_fname = r".\CapturesSkew\2019.3.28_13.37.2_VNOM\CRB_Traffic_DigGainEn__module_"
               
for ppm in [0,50,100]:
    for module in [1,2]:
        for type in ['DSP','ADC','AFE']:
            fnames=[]
            for skew in [127,0,248]:
                fnames.append(prefix_fname+str(module)+"_ppm_"+str(ppm)+\
                              "__skewStart_"+str(skew)+"_"+str(type)+str(TS)+".npy")

            fname_out=prefix_fname+str(module)+"_ppm_"+str(ppm)+\
                              "_"+str(type)+str(TS)+".png"
            datasets = cbToData(fnames,fname_out, shuffle=1)
            plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=False)  
            plt.clf()
            plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=True)            
            plt.clf()


In [ ]:
TS='_2019.3.28_2.17.57'
prefix_fname = r".\CapturesSkew\2019.3.28_2.17.57_VNOM\CRB_Traffic_module_"
               
for ppm in [0,50,100]:
    for module in [1,2]:
        for type in ['DSP','ADC','AFE']:
            fnames=[]
            for skew in [0,127,248]:
                fnames.append(prefix_fname+str(module)+"_ppm_"+str(ppm)+\
                              "__skewStart_"+str(skew)+"_"+str(type)+str(TS)+".npy")

            fname_out=prefix_fname+str(module)+"_ppm_"+str(ppm)+\
                              "_"+str(type)+str(TS)+".png"
            
            datasets = cbToData(fnames,fname_out, shuffle=1)
            plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=False)  
            plt.clf()
            plotCaptureBuffer(datasets, fnames,fname_out, shuffle = 1, log=True)            
            plt.clf()

In [ ]:
'''
import matplotlib.tri as tri
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d.art3d as art3d
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

def plotCaptureBuffer(fnames,fname_out, shuffle = 1):
    datasets=[]

    for fname in fnames:
        DATA=np.load(fname)
        DATA64=[DATA[i::64] for i in range(64)]
        x=np.linspace(0,63,64)
        y=np.linspace(0,len(DATA64[0]),len(DATA64[0])+1)

        if shuffle == 0:
            interleave = [i for i in range(0,64)]

        elif shuffle == 1:
            interleave = [8*(i%8)+int(i/8) for i in range(64)] #EVEN_I, EVEN_Q, EVEN_IB, EVEN_QB, ODD_I, ODD_Q, ODD_IB, ODD_QB
            EVEN_I =  interleave[0 : 8]
            EVEN_Q =  interleave[8 :16]
            EVEN_IB = interleave[16:24]
            EVEN_QB = interleave[24:32]
            ODD_I =   interleave[32:40]
            ODD_Q =   interleave[40:48]
            ODD_IB =  interleave[48:56]
            ODD_QB =  interleave[56:64]
            #interleave = EVEN_I + EVEN_Q + EVEN_IB + EVEN_QB + ODD_I + ODD_Q + ODD_IB + ODD_QB
            interleave = EVEN_I + ODD_I + EVEN_Q + ODD_Q + EVEN_IB + ODD_IB + EVEN_QB + ODD_QB
        elif shuffle ==2:
            interleave = [8*(i%8)+int(i/8) for i in range(64)] #EVEN_I, EVEN_Q, EVEN_IB, EVEN_QB, ODD_I, ODD_Q, ODD_IB, ODD_QB
            EVEN_I =  interleave[0 : 8:2] + interleave[0 +1: 8:2]
            EVEN_Q =  interleave[8 :16:2] + interleave[8 +1:16:2]
            EVEN_IB = interleave[16:24:2] + interleave[16+1:24:2]
            EVEN_QB = interleave[24:32:2] + interleave[24+1:32:2]
            ODD_I =   interleave[32:40:2] + interleave[32+1:40:2]
            ODD_Q =   interleave[40:48:2] + interleave[40+1:48:2]
            ODD_IB =  interleave[48:56:2] + interleave[48+1:56:2]
            ODD_QB =  interleave[56:64:2] + interleave[56+1:64:2]
            #interleave = EVEN_I + EVEN_Q + EVEN_IB + EVEN_QB + ODD_I + ODD_Q + ODD_IB + ODD_QB
            interleave = EVEN_I + ODD_I + EVEN_Q + ODD_Q + EVEN_IB + ODD_IB + EVEN_QB + ODD_QB
        elif shuffle ==3:
            interleave = [i for i in range(0,64,2)]+[i for i in range(0+1,64,2)]
        elif shuffle ==4:
            interleave = [i for i in range(0,64,2)]+[i for i in range(0+1,64,2)]
            EVEN_I =  interleave[ 0+0:32:2]
            EVEN_Q =  interleave[ 0+1:32:2]
            EVEN_IB = interleave[ 0+2:32:2]
            EVEN_QB = interleave[ 0+3:32:2]
            ODD_I =   interleave[32+0:64:2]
            ODD_Q =   interleave[32+1:64:2]
            ODD_IB =  interleave[32+2:64:2]
            ODD_QB =  interleave[32+3:64:2]
            interleave = EVEN_I + EVEN_Q + EVEN_IB + EVEN_QB + ODD_I + ODD_Q + ODD_IB + ODD_QB
        DATA1=[]
        for i in range(64):
            xx=[i]*len(DATA64[0])
            DATA1+=([[each[0],each[1],each[2],int(each[0]/8)] for each in zip(xx,y,DATA64[interleave[i]])])

        DATA1=np.array(DATA1)

        dataset = pd.DataFrame({'SubADC':DATA1[:,0],'Time':DATA1[:,1],'Sample':DATA1[:,2],'SubADC_Group':DATA1[:,3]})
        dataset1=dataset.groupby(['Sample','SubADC']).count().reset_index()
        datasets.append(dataset1)

    fig = plt.figure(figsize=(32,18))
    plt.tight_layout() 
    
    max_z=0
    for i in range(len(datasets)):
        max_z = max(max_z,max(datasets[i]['Time'].values))
    
    count=0
    for DATASET in datasets:
    
        ax = fig.add_subplot(1,len(datasets),count+1)
    
        x=DATASET['SubADC'].values
        y=DATASET['Sample'].values
        z=DATASET['Time'].values
    
        # Now create the Triangulation.
        # (Creating a Triangulation without specifying the triangles results in the
        # Delaunay triangulation of the points.)
        triang = tri.Triangulation(x, y)
    
        # First create the x and y coordinates of the points.
        n_angles = 20
        n_radii = 10
        min_radius = 0.15
        radii = np.linspace(min_radius, 0.95, n_radii)
    
        angles = np.linspace(0, 2 * np.pi, n_angles, endpoint=False)
        angles = np.repeat(angles[..., np.newaxis], n_radii, axis=1)
        angles[:, 1::2] += np.pi / n_angles
    
        # Mask off unwanted triangles.
        triang.set_mask(np.hypot(x[triang.triangles].mean(axis=1),
                                 y[triang.triangles].mean(axis=1))
                        < min_radius)
    
        #-----------------------------------------------------------------------------
        # Refine data
        #-----------------------------------------------------------------------------
        refiner = tri.UniformTriRefiner(triang)
        tri_refi, z_test_refi = refiner.refine_field(z, subdiv=3)
    
        #-----------------------------------------------------------------------------
        # Plot the triangulation and the high-res iso-contours
        #-----------------------------------------------------------------------------
    
        #ax.set_aspect('equal')
        ax.triplot(triang, lw=0.3, color='white',alpha=.5)
    
        levels = np.arange(0., max_z, 2)
        cmap = cm.get_cmap(name='jet', lut=None)
        z_test_refi = [each if each>0 else 0 for each in z_test_refi]
        surf = ax.tricontourf(tri_refi, z_test_refi, levels=levels, cmap=cmap)
        
        #ax.tricontour(tri_refi, z_test_refi, levels=levels,
        #               colors=['0.25', '0.5', '0.5', '0.5', '0.5'],
        #               linewidths=[.2, 0.1, 0.1, 0.1, 0.1])
    
        ax.set_title('Capture Buffer')
        ax.set_xlabel('SubADC_Group')
        ax.set_ylabel('Sample '+fnames[count])
        ax.set_ylim(-128,128)
    
        plt.plot([16,16], [-128, 128], 'w-.', lw=1)
        plt.plot([32,32], [-128, 128], 'w-.', lw=1)
        plt.plot([48,48], [-128, 128], 'w-.', lw=1)
        #c = fig.colorbar(surf, shrink=1, aspect=3, orientation='vertical')
        count+=1
    
    
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(surf, shrink=1, aspect=3, orientation='vertical', cax=cbar_ax)
    
    plt.savefig(fname_out)
'''

'''
def plotCaptureBuffer2(datasets, fnames,fname_out, shuffle = 1):
    fig = plt.figure(figsize=(32,18))
    plt.tight_layout() 
    
    max_z=0
    for i in range(len(datasets)):
        max_z = max(max_z,max(datasets[i]['Time'].values))
    
    count=0
    outer = gridspec.GridSpec(1, len(datasets), wspace=0.2, hspace=0.2)
    #outer = gridspec.GridSpec(nrows=2, ncols=len(datasets), width_ratios=[1]*len(datasets),height_ratios=[1,9])
    for DATASET in datasets:
        #ax_out = fig.add_subplot(outer[0,count])
        ##ax_out.set_ylabel('Sample '+fnames[count])
        #ax_out.set_title('Sample '+fnames[count])
        #ax_out.tick_params(labelcolor=(1.,1.,1., 0.0), top='off', bottom='off', left='off', right='off')
        #ax_out._frameon = False
 
        inner = gridspec.GridSpecFromSubplotSpec(1, 9,
                        subplot_spec=outer[count], wspace=0.1, hspace=0.1, width_ratios=[0.25]+[1]*8)
        ax = fig.add_subplot(inner[0])
        ax.set_ylabel('Sample '+fnames[count])
        ax.tick_params(labelcolor=(1.,1.,1., 0.0), top=False, bottom=False, left=False, right=False)
        ax._frameon = False
        
        for set in range(8):
            
            ax = fig.add_subplot(inner[set+1])
            #df.loc[df['shield'] > 6]
            DATASET1 = DATASET.loc[DATASET['SubADC'].isin([i for i in range(set*8,(set+1)*8)])].copy()
            DATASET1.reset_index()
            x=DATASET1['SubADC'].values
            y=DATASET1['Sample'].values
            z=DATASET1['Time'].values

            # Now create the Triangulation.
            # (Creating a Triangulation without specifying the triangles results in the
            # Delaunay triangulation of the points.)
            triang = tri.Triangulation(x, y)

            # First create the x and y coordinates of the points.
            n_angles = 20
            n_radii = 10
            min_radius = 0.15
            radii = np.linspace(min_radius, 0.95, n_radii)

            angles = np.linspace(0, 2 * np.pi, n_angles, endpoint=False)
            angles = np.repeat(angles[..., np.newaxis], n_radii, axis=1)
            angles[:, 1::2] += np.pi / n_angles

            # Mask off unwanted triangles.
            triang.set_mask(np.hypot(x[triang.triangles].mean(axis=1),
                                     y[triang.triangles].mean(axis=1))
                            < min_radius)

            #-----------------------------------------------------------------------------
            # Refine data
            #-----------------------------------------------------------------------------
            refiner = tri.UniformTriRefiner(triang)
            tri_refi, z_test_refi = refiner.refine_field(z, subdiv=3)

            #-----------------------------------------------------------------------------
            # Plot the triangulation and the high-res iso-contours
            #-----------------------------------------------------------------------------

            #ax.set_aspect('equal')
            ax.triplot(triang, lw=0.3, color='white',alpha=.5)

            levels = np.arange(0., max_z, 2)
            cmap = cm.get_cmap(name='jet', lut=None)
            z_test_refi = [each if each>0 else 0 for each in z_test_refi]
            surf = ax.tricontourf(tri_refi, z_test_refi, levels=levels, cmap=cmap)

            #ax.tricontour(tri_refi, z_test_refi, levels=levels,
            #               colors=['0.25', '0.5', '0.5', '0.5', '0.5'],
            #               linewidths=[.2, 0.1, 0.1, 0.1, 0.1])

            ax.set_ylim(-128,128)
            
            ax.set_xticks([])
            if set !=0:
                ax.set_yticks([])

            #plt.plot([16,16], [-128, 128], 'w-.', lw=1)
            #plt.plot([32,32], [-128, 128], 'w-.', lw=1)
            #plt.plot([48,48], [-128, 128], 'w-.', lw=1)
            #c = fig.colorbar(surf, shrink=1, aspect=3, orientation='vertical')
        count+=1
    
    
    fig.subplots_adjust(right=0.8)
    cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
    fig.colorbar(surf, shrink=1, aspect=3, orientation='vertical', cax=cbar_ax)
    
    plt.savefig(fname_out)
'''

In [ ]:
'''
from matplotlib.colors import LogNorm

def plotCaptureBuffer3(datasets, fnames,fname_out, shuffle = 1, log=True):
    fig = plt.figure(figsize=(32,18))
    plt.tight_layout() 
    
    max_z=0
    for i in range(len(datasets)):
        max_z = max(max_z,max(datasets[i]['Time'].values))
    
    count=0
    outer = gridspec.GridSpec(1, len(datasets)+1, wspace=0.2, hspace=0.2, width_ratios=[1]*len(datasets)+[.05])
    
    for DATASET in datasets:
        z=DATASET['Time']
        zmax=np.amax(z)

    for DATASET in datasets:
        inner = gridspec.GridSpecFromSubplotSpec(1, 9,
                        subplot_spec=outer[count], wspace=0.05, hspace=0.1, width_ratios=[0.25]+[1]*8)
        ax = fig.add_subplot(inner[0])
        ax.set_ylabel('Sample '+fnames[count])
        ax.tick_params(labelcolor=(1.,1.,1., 0.0), top=False, bottom=False, left=False, right=False)
        ax._frameon = False

        for set in range(8):
            ax = fig.add_subplot(inner[set+1])
            DATASET1 = DATASET.loc[DATASET['SubADC'].isin([i for i in range(set*8,(set+1)*8)])].copy()
            DATASET1.reset_index()
            x=DATASET1['SubADC']
            y=DATASET1['Sample']
            z=DATASET1['Time']

            z2=np.zeros([256,8])
            for index, row in DATASET1.iterrows():
                z2[int(row['Sample'])+128][int(row['SubADC'])-8*set] = int(row['Time'])

            extent=(0, 8, -128, 128)
            cmap = cm.jet

            cmap = cm.jet
            if log:
                im = plt.imshow(z2, extent=extent, norm=LogNorm(vmin=1e-15, vmax=zmax), cmap=cmap, aspect='auto')
            else:
                for each in range(len(z2)):
                    for each_each in range(len(z2[each])):
                        if int(z2[each][each_each])==0:
                            
                            z2[each][each_each]='NaN' 
                            #print(z2[each][each_each])
                            
                im = plt.imshow(z2, extent=extent, cmap=cmap, aspect='auto')
            if set !=0:
                ax.set_yticks([])
            ax.set_xticks([])
            for i in range(-128,128,8):
                ax.plot([0,8],[i,i],'k-',alpha=.6)

            plt.clim(zmin, zmax)
            ax.set_facecolor('lightgray')
        count+=1
        
    inner = gridspec.GridSpecFromSubplotSpec(1, 1,
                subplot_spec=outer[len(datasets)])
    ax = fig.add_subplot(inner[0])
    
    plt.colorbar(im,fraction=.25, pad=0.1,ax=None,cax=ax)
    plt.savefig(fname_out.split('.png')[0]+'_shuffle_'+str(shuffle)+'_log_'+str(log)+'.png')
'''